In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)



In [2]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])
a

b = a.flatten()
b
c = a.flatten('F')
c

array([[1, 2, 3],
       [4, 5, 6]])

array([1, 2, 3, 4, 5, 6])

array([1, 4, 2, 5, 3, 6])

In [3]:
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import numpy as np
from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_name, drivers_by_id

# DriverSet = OrderedSet(
#     [
#         'wang-cheng',
#         'li-changlong',
#         'chen-hongmei',
#         'zheng-longfei',
#     ]
# )
# DriverSet[0]
# DriverSet.get_loc('wang-cheng')
# DriverSet


Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): <class 'eos.data_io.struct.eos_data.RecordDoc'>; Concrete Class: (eos.data_io.pool.pool.Pool[eos.data_io.struct.eos_data.RecordDoc],)
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.buffer.buffer.Buffer[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.buffer.buffer.Buffer[~DocItemT], typing.Generic[~DocItemT])


2023-06-12 17:32:30.793014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

ts = pd.to_datetime(datetime.now())
ts_ind = ts + pd.to_timedelta(np.arange(2), 'H')
ts_ind
df = pd.DataFrame(a, index=ts_ind, columns=['c1', 'c2', 'c3'])
df
df.index[0]


DatetimeIndex(['2023-06-12 17:32:31.865217', '2023-06-12 18:32:31.865217'], dtype='datetime64[ns]', freq=None)

,c1,c2,c3
2023-06-12 17:32:31.865217,1,2,3
2023-06-12 18:32:31.865217,4,5,6


Timestamp('2023-06-12 17:32:31.865217')

In [5]:
df_ts = pd.DataFrame(np.array([ts]), columns=['timestamp'])
df_ts

,timestamp
0,2023-06-12 17:32:31.865217


In [6]:
s = np.arange(10)
s

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Generate state

In [7]:
ss = np.arange(12)
a1 = ss[:4].tolist()
a2 = ss[4:8].tolist()
a3 = ss[8:].tolist()
ss = [a1, a2, a3]
ss
df_s = pd.DataFrame(ss, columns=['t', 'v', 'p', 'b'])  # .set_index('t')
df_s

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]

,t,v,p,b
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [8]:
np.arange(0, 4 * 20, 20)

array([ 0, 20, 40, 60])

In [9]:
ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3}
).set_index('timestep')
df_ss.columns.name = 'tuple'
df_ss


DatetimeIndex(['2023-06-12 17:32:31.865217', '2023-06-12 17:32:31.885217',
               '2023-06-12 17:32:31.905217', '2023-06-12 17:32:31.925217'],
              dtype='datetime64[ns]', freq=None)

tuple,velocity,thrust,brake
timestep,,,
2023-06-12 17:32:31.865217,0,4,8
2023-06-12 17:32:31.885217,1,5,9
2023-06-12 17:32:31.905217,2,6,10
2023-06-12 17:32:31.925217,3,7,11


In [10]:
state = df_ss.stack().swaplevel(0, 1)
state.name = 'state'
state.sort_index(inplace=True)
state

tuple     timestep                  
brake     2023-06-12 17:32:31.865217     8
          2023-06-12 17:32:31.885217     9
          2023-06-12 17:32:31.905217    10
          2023-06-12 17:32:31.925217    11
thrust    2023-06-12 17:32:31.865217     4
          2023-06-12 17:32:31.885217     5
          2023-06-12 17:32:31.905217     6
          2023-06-12 17:32:31.925217     7
velocity  2023-06-12 17:32:31.865217     0
          2023-06-12 17:32:31.885217     1
          2023-06-12 17:32:31.905217     2
          2023-06-12 17:32:31.925217     3
Name: state, dtype: int64

In [11]:
# from functools import reduce
#
#
# velocity = pd.Series(ss[:4], name='velocity')
# thrust = pd.Series(ss[4:8], name='thrust')
# brake = pd.Series(ss[8:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)


In [12]:
# from functools import reduce
#
#
# velocity = pd.Series(s[:4], name='velocity')
# thrust = pd.Series(s[4:7], name='thrust')
# brake = pd.Series(s[7:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)
#
#

In [13]:
row_ind = np.arange(50, 80, 10)
# row_ind
ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), 'ms')
pedal_ind = np.linspace(0, 1.0, 5)
# pedal_ind
# action_ind = pd.MultiIndex.from_product([row_ind, ts_ind], names=['row', 'timestamp'])
a = len(state) + np.arange(15)
# a
action = (
    pd.DataFrame(
        {
            'pedal': pedal_ind,
            row_ind[0]: a[:5],
            row_ind[1]: a[5:10],
            row_ind[2]: a[10:],
        }
    ).set_index('pedal')
    # .stack()
    # .swaplevel(0, 1)
    .sort_index()
)
action.columns
action.columns = pd.MultiIndex.from_arrays(
    [action.columns, ts_ind], names=['nominalspeed', 'timestep']
)
action = action.stack([0, 1]).swaplevel(0, 1).swaplevel(1, 2).sort_index()
action.name = 'action'
# action.index.names = ['nominalspeed', 'timestamp']
# action.index
action

Index([50, 60, 70], dtype='object')

nominalspeed  timestep                    pedal
50            2023-06-12 17:32:31.965217  0.00     12.0
                                          0.25     13.0
                                          0.50     14.0
                                          0.75     15.0
                                          1.00     16.0
60            2023-06-12 17:32:31.985217  0.00     17.0
                                          0.25     18.0
                                          0.50     19.0
                                          0.75     20.0
                                          1.00     21.0
70            2023-06-12 17:32:32.005217  0.00     22.0
                                          0.25     23.0
                                          0.50     24.0
                                          0.75     25.0
                                          1.00     26.0
Name: action, dtype: float64

In [14]:
# a = len(state) + np.arange(6)
# ar0 = pd.Series(a[:2], name='row0')
# ar1 = pd.Series(a[2:4], name='row1')
# ar2 = pd.Series(a[4:], name='row2')
# series = [ar0, ar1, ar2]
# action = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# action = action.stack().swaplevel(0, 1).sort_index()
# action.name = 'action'
# action
# len(action) + len(state)

In [15]:
reward = pd.Series([len(state) + len(action)], name='reward')
# reward.name = 'reward'

In [16]:
# velocity = pd.Series(len(state) + len(action) + len(reward) + s[:4], name='velocity')
# thrust = pd.Series(len(state) + len(action) + len(reward) + s[4:7], name='thrust')
# brake = pd.Series(len(state) + len(action) + len(reward) + s[7:], name='brake')
# series = [velocity, thrust, brake]
# nstate = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# nstate = nstate.stack().swaplevel(0, 1).sort_index()
# nstate.name = 'nstate'
# nstate
# len(nstate)
ts_ind = ts + pd.to_timedelta(5, 's') + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ts_ind
ss = np.arange(12) + len(state) + len(action) + len(reward)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
nstate = (
    pd.DataFrame({'timestamp': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3})
    .set_index('timestamp')
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = 'nstate'
nstate.index.names = ['tuple', 'timestep']
len(nstate)
nstate

DatetimeIndex(['2023-06-12 17:32:36.865217', '2023-06-12 17:32:36.885217',
               '2023-06-12 17:32:36.905217', '2023-06-12 17:32:36.925217'],
              dtype='datetime64[ns]', freq=None)

12

tuple     timestep                  
brake     2023-06-12 17:32:36.865217    36
          2023-06-12 17:32:36.885217    37
          2023-06-12 17:32:36.905217    38
          2023-06-12 17:32:36.925217    39
thrust    2023-06-12 17:32:36.865217    32
          2023-06-12 17:32:36.885217    33
          2023-06-12 17:32:36.905217    34
          2023-06-12 17:32:36.925217    35
velocity  2023-06-12 17:32:36.865217    28
          2023-06-12 17:32:36.885217    29
          2023-06-12 17:32:36.905217    30
          2023-06-12 17:32:36.925217    31
Name: nstate, dtype: int64

In [17]:
timestamp = pd.Series([ts], name='timestamp')
timestamp.index
state.index
action.index
reward.index
nstate.index

RangeIndex(start=0, stop=1, step=1)

MultiIndex([(   'brake', '2023-06-12 17:32:31.865217'),
            (   'brake', '2023-06-12 17:32:31.885217'),
            (   'brake', '2023-06-12 17:32:31.905217'),
            (   'brake', '2023-06-12 17:32:31.925217'),
            (  'thrust', '2023-06-12 17:32:31.865217'),
            (  'thrust', '2023-06-12 17:32:31.885217'),
            (  'thrust', '2023-06-12 17:32:31.905217'),
            (  'thrust', '2023-06-12 17:32:31.925217'),
            ('velocity', '2023-06-12 17:32:31.865217'),
            ('velocity', '2023-06-12 17:32:31.885217'),
            ('velocity', '2023-06-12 17:32:31.905217'),
            ('velocity', '2023-06-12 17:32:31.925217')],
           names=['tuple', 'timestep'])

MultiIndex([(50, '2023-06-12 17:32:31.965217',  0.0),
            (50, '2023-06-12 17:32:31.965217', 0.25),
            (50, '2023-06-12 17:32:31.965217',  0.5),
            (50, '2023-06-12 17:32:31.965217', 0.75),
            (50, '2023-06-12 17:32:31.965217',  1.0),
            (60, '2023-06-12 17:32:31.985217',  0.0),
            (60, '2023-06-12 17:32:31.985217', 0.25),
            (60, '2023-06-12 17:32:31.985217',  0.5),
            (60, '2023-06-12 17:32:31.985217', 0.75),
            (60, '2023-06-12 17:32:31.985217',  1.0),
            (70, '2023-06-12 17:32:32.005217',  0.0),
            (70, '2023-06-12 17:32:32.005217', 0.25),
            (70, '2023-06-12 17:32:32.005217',  0.5),
            (70, '2023-06-12 17:32:32.005217', 0.75),
            (70, '2023-06-12 17:32:32.005217',  1.0)],
           names=['nominalspeed', 'timestep', 'pedal'])

RangeIndex(start=0, stop=1, step=1)

MultiIndex([(   'brake', '2023-06-12 17:32:36.865217'),
            (   'brake', '2023-06-12 17:32:36.885217'),
            (   'brake', '2023-06-12 17:32:36.905217'),
            (   'brake', '2023-06-12 17:32:36.925217'),
            (  'thrust', '2023-06-12 17:32:36.865217'),
            (  'thrust', '2023-06-12 17:32:36.885217'),
            (  'thrust', '2023-06-12 17:32:36.905217'),
            (  'thrust', '2023-06-12 17:32:36.925217'),
            ('velocity', '2023-06-12 17:32:36.865217'),
            ('velocity', '2023-06-12 17:32:36.885217'),
            ('velocity', '2023-06-12 17:32:36.905217'),
            ('velocity', '2023-06-12 17:32:36.925217')],
           names=['tuple', 'timestep'])

In [80]:
timestamp_index = (timestamp.name, '', 0, '')
timestamp_index
state_index = [(state.name, *i, '') for i in state.index]
state_index
reward_index = (reward.name, '', ts_ind[0], '')
reward_index
action_index = [(action.name, *i) for i in action.index]
action_index
nstate_index = [(nstate.name, *i, '') for i in nstate.index]
nstate_index

multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, reward_index, *action_index, *nstate_index]
)
multiindex


('timestamp', '', 0, '')

[('state', 'brake', Timestamp('2023-06-12 17:32:31.865217'), ''),
 ('state', 'brake', Timestamp('2023-06-12 17:32:31.885217'), ''),
 ('state', 'brake', Timestamp('2023-06-12 17:32:31.905217'), ''),
 ('state', 'brake', Timestamp('2023-06-12 17:32:31.925217'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:32:31.865217'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:32:31.885217'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:32:31.905217'), ''),
 ('state', 'thrust', Timestamp('2023-06-12 17:32:31.925217'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:32:31.865217'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:32:31.885217'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:32:31.905217'), ''),
 ('state', 'velocity', Timestamp('2023-06-12 17:32:31.925217'), '')]

('reward', '', Timestamp('2023-06-12 17:32:36.865217'), '')

[('action', 50, Timestamp('2023-06-12 17:32:31.965217'), 0.0),
 ('action', 50, Timestamp('2023-06-12 17:32:31.965217'), 0.25),
 ('action', 50, Timestamp('2023-06-12 17:32:31.965217'), 0.5),
 ('action', 50, Timestamp('2023-06-12 17:32:31.965217'), 0.75),
 ('action', 50, Timestamp('2023-06-12 17:32:31.965217'), 1.0),
 ('action', 60, Timestamp('2023-06-12 17:32:31.985217'), 0.0),
 ('action', 60, Timestamp('2023-06-12 17:32:31.985217'), 0.25),
 ('action', 60, Timestamp('2023-06-12 17:32:31.985217'), 0.5),
 ('action', 60, Timestamp('2023-06-12 17:32:31.985217'), 0.75),
 ('action', 60, Timestamp('2023-06-12 17:32:31.985217'), 1.0),
 ('action', 70, Timestamp('2023-06-12 17:32:32.005217'), 0.0),
 ('action', 70, Timestamp('2023-06-12 17:32:32.005217'), 0.25),
 ('action', 70, Timestamp('2023-06-12 17:32:32.005217'), 0.5),
 ('action', 70, Timestamp('2023-06-12 17:32:32.005217'), 0.75),
 ('action', 70, Timestamp('2023-06-12 17:32:32.005217'), 1.0)]

[('nstate', 'brake', Timestamp('2023-06-12 17:32:36.865217'), ''),
 ('nstate', 'brake', Timestamp('2023-06-12 17:32:36.885217'), ''),
 ('nstate', 'brake', Timestamp('2023-06-12 17:32:36.905217'), ''),
 ('nstate', 'brake', Timestamp('2023-06-12 17:32:36.925217'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:32:36.865217'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:32:36.885217'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:32:36.905217'), ''),
 ('nstate', 'thrust', Timestamp('2023-06-12 17:32:36.925217'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:32:36.865217'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:32:36.885217'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:32:36.905217'), ''),
 ('nstate', 'velocity', Timestamp('2023-06-12 17:32:36.925217'), '')]

MultiIndex([('timestamp',         '',                          0,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.865217,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.885217,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.905217,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.925217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.865217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.885217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.905217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.925217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.865217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.885217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.905217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.925217,   ''),
            (   'reward',         '', 2023-06-12 17

In [81]:
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex
observation
observation.dtype

timestamp            0                                   2023-06-12 17:32:31.865217
state      brake     2023-06-12 17:32:31.865217                                   8
                     2023-06-12 17:32:31.885217                                   9
                     2023-06-12 17:32:31.905217                                  10
                     2023-06-12 17:32:31.925217                                  11
           thrust    2023-06-12 17:32:31.865217                                   4
                     2023-06-12 17:32:31.885217                                   5
                     2023-06-12 17:32:31.905217                                   6
                     2023-06-12 17:32:31.925217                                   7
           velocity  2023-06-12 17:32:31.865217                                   0
                     2023-06-12 17:32:31.885217                                   1
                     2023-06-12 17:32:31.905217                             

dtype('O')

In [82]:

observation0 = observation.copy()
observation0.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(1, 'h')
observation1 = observation.copy()
observation1.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(2, 'h')
observation2 = observation.copy()
observation2.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(3, 'h')
observation3 = observation.copy()
observation3.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(4, 'h')
observation4 = observation.copy()
observation4.loc['timestamp', '', 0, ''] = ts + pd.Timedelta(5, 'h')
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]


In [83]:

dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ['tuple', 'measurement', 'timestep', 'extra']
dfs_epi.columns
display('dfs_epi')


MultiIndex([('timestamp',         '',                          0,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.865217,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.885217,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.905217,   ''),
            (    'state',    'brake', 2023-06-12 17:32:31.925217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.865217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.885217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.905217,   ''),
            (    'state',   'thrust', 2023-06-12 17:32:31.925217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.865217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.885217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.905217,   ''),
            (    'state', 'velocity', 2023-06-12 17:32:31.925217,   ''),
            (   'reward',         '', 2023-06-12 17

dfs_epi
tuple                        timestamp                      state   
measurement                                                 brake   
timestep                             0 2023-06-12 17:32:31.865217   
extra                                                               
0           2023-06-12 18:32:31.865217                          8  \
1           2023-06-12 19:32:31.865217                          8   
2           2023-06-12 20:32:31.865217                          8   
3           2023-06-12 21:32:31.865217                          8   
4           2023-06-12 22:32:31.865217                          8   

tuple                                                               
measurement                                                         
timestep    2023-06-12 17:32:31.885217 2023-06-12 17:32:31.905217   
extra                                                               
0                                    9                         10  \
1                                    9                         10   
2                                    9                         10   
3                                    9                         10   
4                                    9                         10   

tuple                                                               
measurement                                                thrust   
timestep    2023-06-12 17:32:31.925217 2023-06-12 17:32:31.865217   
extra                                                               
0                                   11                          4  \
1                                   11                          4   
2                                   11                          4   
3                                   11                          4   
4                                   11                          4   

tuple                                                               
measurement                                                         
timestep    2023-06-12 17:32:31.885217 2023-06-12 17:32:31.905217   
extra                                                               
0                                    5                          6  \
1                                    5                          6   
2                                    5                          6   
3                                    5                          6   
4                                    5                          6   

tuple                                                              ...   
measurement                                              velocity  ...   
timestep    2023-06-12 17:32:31.925217 2023-06-12 17:32:31.865217  ...   
extra                                                              ...   
0                                    7                          0  ...  \
1                                    7                          0  ...   
2                                    7                          0  ...   
3                                    7                          0  ...   
4                                    7                          0  ...   

tuple                           nstate                              
measurement                      brake                              
timestep    2023-06-12 17:32:36.905217 2023-06-12 17:32:36.925217   
extra                                                               
0                                   38                         39  \
1                                   38                         39   
2                                   38                         39   
3                                   38                         39   
4                                   38                         39   

tuple                                                               
measurement                     thrust                              
timestep    2023-06-12 17:32:36.865217 2023-06-12 17:32:36.885217   
extra   

In [84]:
dfs_episode1 = dfs_epi.copy().sort_index(axis=1)
dfs_episode1.set_index(('timestamp', '', 0, ''), inplace=True)
dfs_episode1.index.name = 'timestamp'
idx = pd.IndexSlice
dfs_episode1

tuple                                          action                           
measurement                                        50                           
timestep                   2023-06-12 17:32:31.965217                           
extra                                             0.0  0.25   0.5  0.75   1.0   
timestamp                                                                       
2023-06-12 18:32:31.865217                       13.0  14.0  15.0  16.0  17.0  \
2023-06-12 19:32:31.865217                       13.0  14.0  15.0  16.0  17.0   
2023-06-12 20:32:31.865217                       13.0  14.0  15.0  16.0  17.0   
2023-06-12 21:32:31.865217                       13.0  14.0  15.0  16.0  17.0   
2023-06-12 22:32:31.865217                       13.0  14.0  15.0  16.0  17.0   

tuple                                                                           
measurement                                        60                           
timestep                   2023-06-12 17:32:31.985217                           
extra                                             0.0  0.25   0.5  0.75   1.0   
timestamp                                                                       
2023-06-12 18:32:31.865217                       18.0  19.0  20.0  21.0  22.0  \
2023-06-12 19:32:31.865217                       18.0  19.0  20.0  21.0  22.0   
2023-06-12 20:32:31.865217                       18.0  19.0  20.0  21.0  22.0   
2023-06-12 21:32:31.865217                       18.0  19.0  20.0  21.0  22.0   
2023-06-12 22:32:31.865217                       18.0  19.0  20.0  21.0  22.0   

tuple                       ...                      state   
measurement                 ...                      brake   
timestep                    ... 2023-06-12 17:32:31.905217   
extra                       ...                              
timestamp                   ...                              
2023-06-12 18:32:31.865217  ...                         10  \
2023-06-12 19:32:31.865217  ...                         10   
2023-06-12 20:32:31.865217  ...                         10   
2023-06-12 21:32:31.865217  ...                         10   
2023-06-12 22:32:31.865217  ...                         10   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.925217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                         11  \
2023-06-12 19:32:31.865217                         11   
2023-06-12 20:32:31.865217                         11   
2023-06-12 21:32:31.865217                         11   
2023-06-12 22:32:31.865217                         11   

tuple                                                   
measurement                                    thrust   
timestep                   2023-06-12 17:32:31.865217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                          4  \
2023-06-12 19:32:31.865217                          4   
2023-06-12 20:32:31.865217                          4   
2023-06-12 21:32:31.865217                          4   
2023-06-12 22:32:31.865217                          4   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.885217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                          5  \
2023-06-12 19:32:31.865217                          5   
2023-06-12 20:32:31.865217                          5   
2023-06-12 21:32:31.865217                          5   
2023-06-12 22:32:31.865217                          5   

tuple                                       

In [85]:
dfs_episode1.loc[:, idx['action':'state', :, :, :]] = dfs_episode1.loc[
    :, idx['action':'state', :, :, :]
].astype('int')
dfs_episode1
dfs_episode1.dtypes
dfs_episode1.columns

tuple                                          action                     
measurement                                        50                     
timestep                   2023-06-12 17:32:31.965217                     
extra                                             0.0 0.25 0.5 0.75 1.0   
timestamp                                                                 
2023-06-12 18:32:31.865217                         13   14  15   16  17  \
2023-06-12 19:32:31.865217                         13   14  15   16  17   
2023-06-12 20:32:31.865217                         13   14  15   16  17   
2023-06-12 21:32:31.865217                         13   14  15   16  17   
2023-06-12 22:32:31.865217                         13   14  15   16  17   

tuple                                                                    ...   
measurement                                        60                    ...   
timestep                   2023-06-12 17:32:31.985217                    ...   
extra                                             0.0 0.25 0.5 0.75 1.0  ...   
timestamp                                                                ...   
2023-06-12 18:32:31.865217                         18   19  20   21  22  ...  \
2023-06-12 19:32:31.865217                         18   19  20   21  22  ...   
2023-06-12 20:32:31.865217                         18   19  20   21  22  ...   
2023-06-12 21:32:31.865217                         18   19  20   21  22  ...   
2023-06-12 22:32:31.865217                         18   19  20   21  22  ...   

tuple                                           state   
measurement                                     brake   
timestep                   2023-06-12 17:32:31.905217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                         10  \
2023-06-12 19:32:31.865217                         10   
2023-06-12 20:32:31.865217                         10   
2023-06-12 21:32:31.865217                         10   
2023-06-12 22:32:31.865217                         10   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.925217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                         11  \
2023-06-12 19:32:31.865217                         11   
2023-06-12 20:32:31.865217                         11   
2023-06-12 21:32:31.865217                         11   
2023-06-12 22:32:31.865217                         11   

tuple                                                   
measurement                                    thrust   
timestep                   2023-06-12 17:32:31.865217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                          4  \
2023-06-12 19:32:31.865217                          4   
2023-06-12 20:32:31.865217                          4   
2023-06-12 21:32:31.865217                          4   
2023-06-12 22:32:31.865217                          4   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.885217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                          5  \
2023-06-12 19:32:31.865217                          5   
2023-06-12 20:32:31.865217                          5   
2023-06-12 21:32:31.865217                          5   
2023-06-12 22:32:31.865217                          5   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.905

tuple   measurement  timestep                    extra
action  50           2023-06-12 17:32:31.965217  0.0      object
                                                 0.25     object
                                                 0.5      object
                                                 0.75     object
                                                 1.0      object
        60           2023-06-12 17:32:31.985217  0.0      object
                                                 0.25     object
                                                 0.5      object
                                                 0.75     object
                                                 1.0      object
        70           2023-06-12 17:32:32.005217  0.0      object
                                                 0.25     object
                                                 0.5      object
                                                 0.75     object
                                   

MultiIndex([('action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ('action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ('action',         50, 2023-06-12 17:32:31.965217,  0.5),
            ('action',         50, 2023-06-12 17:32:31.965217, 0.75),
            ('action',         50, 2023-06-12 17:32:31.965217,  1.0),
            ('action',         60, 2023-06-12 17:32:31.985217,  0.0),
            ('action',         60, 2023-06-12 17:32:31.985217, 0.25),
            ('action',         60, 2023-06-12 17:32:31.985217,  0.5),
            ('action',         60, 2023-06-12 17:32:31.985217, 0.75),
            ('action',         60, 2023-06-12 17:32:31.985217,  1.0),
            ('action',         70, 2023-06-12 17:32:32.005217,  0.0),
            ('action',         70, 2023-06-12 17:32:32.005217, 0.25),
            ('action',         70, 2023-06-12 17:32:32.005217,  0.5),
            ('action',         70, 2023-06-12 17:32:32.005217, 0.75),
            ('action

In [86]:
dfs_episode = dfs_epi.copy()
dfs_episode.set_index(('timestamp', '', 0, ''), inplace=True)
dfs_episode.index.name = 'timestamp'
idx = pd.IndexSlice

# dfs_episode = dfs_episode.astype({idx['state', 'velocity', :]: 'int'})  # Error unhashable type: 'slice' for dictionary
# sorted_idx = **(idx['state', :, :]:'int')
# sorted_idx
dfs_episode['state'] = dfs_episode['state'].astype(
    'float'
)  # float16 not allowed in parquet
dfs_episode['action'] = dfs_episode['action'].astype('float')
dfs_episode['reward'] = dfs_episode['reward'].astype('float')
dfs_episode['nstate'] = dfs_episode['nstate'].astype('float')
dfs_episode
dfs_episode.dtypes
# dfs_episode.columns
# dfs_epi
# dfs_episode


tuple                                           state   
measurement                                     brake   
timestep                   2023-06-12 17:32:31.865217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                        8.0  \
2023-06-12 19:32:31.865217                        8.0   
2023-06-12 20:32:31.865217                        8.0   
2023-06-12 21:32:31.865217                        8.0   
2023-06-12 22:32:31.865217                        8.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.885217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                        9.0  \
2023-06-12 19:32:31.865217                        9.0   
2023-06-12 20:32:31.865217                        9.0   
2023-06-12 21:32:31.865217                        9.0   
2023-06-12 22:32:31.865217                        9.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.905217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                       10.0  \
2023-06-12 19:32:31.865217                       10.0   
2023-06-12 20:32:31.865217                       10.0   
2023-06-12 21:32:31.865217                       10.0   
2023-06-12 22:32:31.865217                       10.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.925217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                       11.0  \
2023-06-12 19:32:31.865217                       11.0   
2023-06-12 20:32:31.865217                       11.0   
2023-06-12 21:32:31.865217                       11.0   
2023-06-12 22:32:31.865217                       11.0   

tuple                                                   
measurement                                    thrust   
timestep                   2023-06-12 17:32:31.865217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                        4.0  \
2023-06-12 19:32:31.865217                        4.0   
2023-06-12 20:32:31.865217                        4.0   
2023-06-12 21:32:31.865217                        4.0   
2023-06-12 22:32:31.865217                        4.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.885217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                        5.0  \
2023-06-12 19:32:31.865217                        5.0   
2023-06-12 20:32:31.865217                        5.0   
2023-06-12 21:32:31.865217                        5.0   
2023-06-12 22:32:31.865217                        5.0   

tuple                                                   
measurement                                             
timestep                   2023-06-12 17:32:31.905217   
extra                                                   
timestamp                                               
2023-06-12 18:32:31.865217                        6.0  \
2023-06-12 19:32:31.865217                        6.0   
2023-06-12 20:32:31.865217                        6.0   
2023-06-12 21:32:31.865217                        6.0   
2023-06-12 22:32:31.865217                        6.0   

tup

tuple   measurement  timestep                    extra
state   brake        2023-06-12 17:32:31.865217           float64
                     2023-06-12 17:32:31.885217           float64
                     2023-06-12 17:32:31.905217           float64
                     2023-06-12 17:32:31.925217           float64
        thrust       2023-06-12 17:32:31.865217           float64
                     2023-06-12 17:32:31.885217           float64
                     2023-06-12 17:32:31.905217           float64
                     2023-06-12 17:32:31.925217           float64
        velocity     2023-06-12 17:32:31.865217           float64
                     2023-06-12 17:32:31.885217           float64
                     2023-06-12 17:32:31.905217           float64
                     2023-06-12 17:32:31.925217           float64
reward               2023-06-12 17:32:36.865217           float64
action  50           2023-06-12 17:32:31.965217  0.0      float64
                     

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [87]:
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id['wang-cheng'].pid], names=['driver']
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_name['VB7'].vid], names=['vehicle']
)

In [88]:

dfs_episode.columns.set_names(
    ['tuple', 'measurement', 'timestep', 'extra'], level=[0, 1, 2, 3], inplace=True
)
dfs_episode
dfs_episode.dtypes


tuple                                                              state   
measurement                                                        brake   
timestep                                      2023-06-12 17:32:31.865217   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:32:31.865217                        8.0  \
                   2023-06-12 19:32:31.865217                        8.0   
                   2023-06-12 20:32:31.865217                        8.0   
                   2023-06-12 21:32:31.865217                        8.0   
                   2023-06-12 22:32:31.865217                        8.0   

tuple                                                                      
measurement                                                                
timestep                                      2023-06-12 17:32:31.885217   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:32:31.865217                        9.0  \
                   2023-06-12 19:32:31.865217                        9.0   
                   2023-06-12 20:32:31.865217                        9.0   
                   2023-06-12 21:32:31.865217                        9.0   
                   2023-06-12 22:32:31.865217                        9.0   

tuple                                                                      
measurement                                                                
timestep                                      2023-06-12 17:32:31.905217   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:32:31.865217                       10.0  \
                   2023-06-12 19:32:31.865217                       10.0   
                   2023-06-12 20:32:31.865217                       10.0   
                   2023-06-12 21:32:31.865217                       10.0   
                   2023-06-12 22:32:31.865217                       10.0   

tuple                                                                      
measurement                                                                
timestep                                      2023-06-12 17:32:31.925217   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:32:31.865217                       11.0  \
                   2023-06-12 19:32:31.865217                       11.0   
                   2023-06-12 20:32:31.865217                       11.0   
                   2023-06-12 21:32:31.865217                       11.0   
                   2023-06-12 22:32:31.865217                       11.0   

tuple                                                                      
measurement                                                       thrust   
timestep                                      2023-06-12 17:32:31.865217   
extra                                                                      
vehicle driver     timestamp                                               
VB7     wang-cheng 2023-06-12 18:32:31.865217                        4.0  \
                   2023-06-12 19:32:31.865217                        4.0   
                   2023-06-12 20:32:31.865217                        4.0   
                   2023-06-12 21:32:31.865217                        4.0   
                   2023-06-12 22:32:31.865217                        4.0   

tuple                                                                      
measurement                                                                
timestep                                   

tuple   measurement  timestep                    extra
state   brake        2023-06-12 17:32:31.865217           float64
                     2023-06-12 17:32:31.885217           float64
                     2023-06-12 17:32:31.905217           float64
                     2023-06-12 17:32:31.925217           float64
        thrust       2023-06-12 17:32:31.865217           float64
                     2023-06-12 17:32:31.885217           float64
                     2023-06-12 17:32:31.905217           float64
                     2023-06-12 17:32:31.925217           float64
        velocity     2023-06-12 17:32:31.865217           float64
                     2023-06-12 17:32:31.885217           float64
                     2023-06-12 17:32:31.905217           float64
                     2023-06-12 17:32:31.925217           float64
reward               2023-06-12 17:32:36.865217           float64
action  50           2023-06-12 17:32:31.965217  0.0      float64
                     

### Convert columns types to int and float16

In [89]:
# idx = pd.IndexSlice
# dfs_episode.loc[:, idx['state', :, :]] = dfs_episode.loc[:, idx['state', :, :]].astype(
#     'int'
# )
# dfs_episode.loc[:, idx['action', :, :]] = dfs_episode.loc[
#     :, idx['action', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['reward', :, :]] = dfs_episode.loc[
#     :, idx['reward', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['nstate', :, :]] = dfs_episode.loc[
#     :, idx['nstate', :, :]
# ].astype('float16')
#
# vel_1 = dfs_episode[[('state', 'velocity', 1)]]
# vel_1.dtypes
# vel_1.index
# vel_1.values
# # type(vel_1)
# vel_1.iloc[0]
# type(vel_1.iloc[0])
# type(vel_1.iloc[0].values[0])

## Add a level of index for episode

In [90]:
episodestart = ts - pd.Timedelta(1, 'h')
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=['episodestart'])
dfs_episode.sort_index(inplace=True)
dfs_episode.index
dfs_episode.columns
dfs_episode

MultiIndex([('2023-06-12 17:32:31.865217', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:32:31.865217', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:32:31.865217', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:32:31.865217', 'VB7', 'wang-cheng', ...),
            ('2023-06-12 17:32:31.865217', 'VB7', 'wang-cheng', ...)],
           names=['episodestart', 'vehicle', 'driver', 'timestamp'])

MultiIndex([( 'state',    'brake', 2023-06-12 17:32:31.865217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.885217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.905217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.925217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.865217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.885217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.905217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.925217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.865217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.885217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.905217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.925217,   ''),
            ('reward',         '', 2023-06-12 17:32:36.865217,   ''),
            ('action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ('action

tuple                                                                                         state   
measurement                                                                                   brake   
timestep                                                                 2023-06-12 17:32:31.865217   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:32:31.865217 VB7     wang-cheng 2023-06-12 18:32:31.865217                        8.0  \
                                              2023-06-12 19:32:31.865217                        8.0   
                                              2023-06-12 20:32:31.865217                        8.0   
                                              2023-06-12 21:32:31.865217                        8.0   
                                              2023-06-12 22:32:31.865217                        8.0   

tuple                                                                                                 
measurement                                                                                           
timestep                                                                 2023-06-12 17:32:31.885217   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:32:31.865217 VB7     wang-cheng 2023-06-12 18:32:31.865217                        9.0  \
                                              2023-06-12 19:32:31.865217                        9.0   
                                              2023-06-12 20:32:31.865217                        9.0   
                                              2023-06-12 21:32:31.865217                        9.0   
                                              2023-06-12 22:32:31.865217                        9.0   

tuple                                                                                                 
measurement                                                                                           
timestep                                                                 2023-06-12 17:32:31.905217   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:32:31.865217 VB7     wang-cheng 2023-06-12 18:32:31.865217                       10.0  \
                                              2023-06-12 19:32:31.865217                       10.0   
                                              2023-06-12 20:32:31.865217                       10.0   
                                              2023-06-12 21:32:31.865217                       10.0   
                                              2023-06-12 22:32:31.865217                       10.0   

tuple                                                                                                 
measurement                                                                                           
timestep                                                                 2023-06-12 17:32:31.925217   
extra                                                                                                 
episodestart               vehicle driver     timestamp                                               
2023-06-12 17:32:31.865217 VB7     wang-cheng 2023-06-12 18:32:31.865217                       11.0  \
                                              2023-06-12 19:32:31.865217                       11.0   
                                              2023-06-12 20:32:31.865217                       11.0   
                                              2023-06-12 21:32:31.865217           

In [91]:
ts_new = pd.to_datetime((datetime.now()))
ts_new
episodestart = ts_new - pd.Timedelta(2, 'd')
episodestart

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level='episodestart')
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks_by_name['VB7'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, 'd')], level='episodestart'
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks_by_name['MP73'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, 'd')], level='episodestart'
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks_by_name['VB7'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, 'd')], level='episodestart'
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks_by_name['MP73'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode0



Timestamp('2023-06-12 18:46:10.575552')

Timestamp('2023-06-10 18:46:10.575552')

tuple                                                                                            state   
measurement                                                                                      brake   
timestep                                                                    2023-06-12 17:32:31.865217   
extra                                                                                                    
episodestart               vehicle driver        timestamp                                               
2023-06-10 18:46:10.575552 VB7     zheng-longfei 2023-06-12 18:32:31.865217                        8.0  \
                                                 2023-06-12 19:32:31.865217                        8.0   
                                                 2023-06-12 20:32:31.865217                        8.0   
                                                 2023-06-12 21:32:31.865217                        8.0   
                                                 2023-06-12 22:32:31.865217                        8.0   

tuple                                                                                                    
measurement                                                                                              
timestep                                                                    2023-06-12 17:32:31.885217   
extra                                                                                                    
episodestart               vehicle driver        timestamp                                               
2023-06-10 18:46:10.575552 VB7     zheng-longfei 2023-06-12 18:32:31.865217                        9.0  \
                                                 2023-06-12 19:32:31.865217                        9.0   
                                                 2023-06-12 20:32:31.865217                        9.0   
                                                 2023-06-12 21:32:31.865217                        9.0   
                                                 2023-06-12 22:32:31.865217                        9.0   

tuple                                                                                                    
measurement                                                                                              
timestep                                                                    2023-06-12 17:32:31.905217   
extra                                                                                                    
episodestart               vehicle driver        timestamp                                               
2023-06-10 18:46:10.575552 VB7     zheng-longfei 2023-06-12 18:32:31.865217                       10.0  \
                                                 2023-06-12 19:32:31.865217                       10.0   
                                                 2023-06-12 20:32:31.865217                       10.0   
                                                 2023-06-12 21:32:31.865217                       10.0   
                                                 2023-06-12 22:32:31.865217                       10.0   

tuple                                                                                                    
measurement                                                                                              
timestep                                                                    2023-06-12 17:32:31.925217   
extra                                                                                                    
episodestart               vehicle driver        timestamp                                               
2023-06-10 18:46:10.575552 VB7     zheng-longfei 2023-06-12 18:32:31.865217                       11.0  \
                                                 2023-06-12 19:32:31.865217                       11.0   
                                                 2023-06-12 20:32:31.865217

In [92]:
# dfs_episode_concat = pd.concat([dfs_episode, dfs_episode0], axis=0)
# display('dfs_episode_concat')
# dfs_episode_concat.index
# dfs_episode_concat.columns

In [93]:
from functools import reduce

episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all[['state', 'action', 'reward', 'nstate']]
display('dfs_episode_all')
dfs_episode_all.index
dfs_episode_all.columns

dfs_episode_all
tuple                                                                                            state   
measurement                                                                                      brake   
timestep                                                                    2023-06-12 17:32:31.865217   
extra                                                                                                    
vehicle driver        episodestart               timestamp                                               
MP73    wang-cheng    2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0  \
                                                 2023-06-12 19:32:31.865217                        8.0   
                                                 2023-06-12 20:32:31.865217                        8.0   
                                                 2023-06-12 21:32:31.865217                        8.0   
                                                 2023-06-12 22:32:31.865217                        8.0   
        zheng-longfei 2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0   
                                                 2023-06-12 19:32:31.865217                        8.0   
                                                 2023-06-12 20:32:31.865217                        8.0   
                                                 2023-06-12 21:32:31.865217                        8.0   
                                                 2023-06-12 22:32:31.865217                        8.0   
VB7     wang-cheng    2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0   
                                                 2023-06-12 19:32:31.865217                        8.0   
                                                 2023-06-12 20:32:31.865217                        8.0   
                                                 2023-06-12 21:32:31.865217                        8.0   
                                                 2023-06-12 22:32:31.865217                        8.0   
                      2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                        8.0   
                                                 2023-06-12 19:32:31.865217                        8.0   
                                                 2023-06-12 20:32:31.865217                        8.0   
                                                 2023-06-12 21:32:31.865217                        8.0   
                                                 2023-06-12 22:32:31.865217                        8.0   
        zheng-longfei 2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0   
                                                 2023-06-12 19:32:31.865217                        8.0   
                                                 2023-06-12 20:32:31.865217                        8.0   
                                                 2023-06-12 21:32:31.865217                        8.0   
                                                 2023-06-12 22:32:31.865217                        8.0   

tuple                                                                                                    
measurement                                                                                              
timestep                                                                    2023-06-12 17:32:31.885217   
extra                                                                                                    
vehicle driver        episodestart               timestamp                                               
MP73    wang-cheng    2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                        9.0  \
                                                 2023-06-12 19:32:31.865217                        9.0   
                                                 2023-06-12 2

MultiIndex([('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', '2023-06-06 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', '2023-06-06 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', '2023-06-06 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', 

MultiIndex([( 'state',    'brake', 2023-06-12 17:32:31.865217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.885217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.905217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.925217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.865217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.885217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.905217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.925217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.865217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.885217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.905217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.925217,   ''),
            ('action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ('action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ('action

## Make MultiIndex to Rows by adding levels of "Vehicle" and "Driver"

###  reset index to move vehicle and driver to columns, preprocessing for dask manipulation

In [94]:
dfs_episodes = dfs_episode_all.reset_index(
    level=['vehicle', 'driver']
)  # unstack vehicle and driver to columns in level 0 with default '' in level 1
# # dfs_episodes.columns
# # dfs_episode0.index = dfs_episode0.index.set_levels([[trucks[0].vid],[drivers[1].pid]], level=['vehicle', 'driver'], verify_integrity=False)
# old_columns = dfs_episodes.columns
# # old_columns[0]= (old_columns[0][0],0)
# new_columns_tuples = [(old_columns[0][0],0), (old_columns[1][0],0)] + old_columns[2:].to_list()
# prepend_columns = pd.MultiIndex.from_tuples(new_columns_tuples, names=old_columns.names)
# # prepend_columns

# dfs_episodes.columns = prepend_columns
dfs_episodes
# dfs_episodes.columns
# dfs_episodes.dtypes
# dfs_episode_all

tuple                                                 vehicle         driver   
measurement                                                                    
timestep                                                                       
extra                                                                          
episodestart               timestamp                                           
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217    MP73     wang-cheng  \
                           2023-06-12 19:32:31.865217    MP73     wang-cheng   
                           2023-06-12 20:32:31.865217    MP73     wang-cheng   
                           2023-06-12 21:32:31.865217    MP73     wang-cheng   
                           2023-06-12 22:32:31.865217    MP73     wang-cheng   
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 19:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 20:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 21:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 22:32:31.865217    MP73  zheng-longfei   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217     VB7     wang-cheng   
                           2023-06-12 19:32:31.865217     VB7     wang-cheng   
                           2023-06-12 20:32:31.865217     VB7     wang-cheng   
                           2023-06-12 21:32:31.865217     VB7     wang-cheng   
                           2023-06-12 22:32:31.865217     VB7     wang-cheng   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217     VB7     wang-cheng   
                           2023-06-12 19:32:31.865217     VB7     wang-cheng   
                           2023-06-12 20:32:31.865217     VB7     wang-cheng   
                           2023-06-12 21:32:31.865217     VB7     wang-cheng   
                           2023-06-12 22:32:31.865217     VB7     wang-cheng   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217     VB7  zheng-longfei   
                           2023-06-12 19:32:31.865217     VB7  zheng-longfei   
                           2023-06-12 20:32:31.865217     VB7  zheng-longfei   
                           2023-06-12 21:32:31.865217     VB7  zheng-longfei   
                           2023-06-12 22:32:31.865217     VB7  zheng-longfei   

tuple                                                                      state   
measurement                                                                brake   
timestep                                              2023-06-12 17:32:31.865217   
extra                                                                              
episodestart               timestamp                                               
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0  \
                           2023-06-12 19:32:31.865217                        8.0   
                           2023-06-12 20:32:31.865217                        8.0   
                           2023-06-12 21:32:31.865217                        8.0   
                           2023-06-12 22:32:31.865217                        8.0   
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0   
                           2023-06-12 19:32:31.865217                        8.0   
                           2023-06-12 20:32:31.865217                        8.0   
                           2023-06-12 21:32:31.865217                        8.0   
                           2023-06-12 22:32:31.865217                        8.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0   
                           2023-06-12 19:32:31.865217                        8.0   
                           2023-06-12 20:32:31.865217                        8.0   
                           2023-06-12 21:32:31.865217                        8.0   
   

In [95]:
dfs_episodes.dtypes

tuple    measurement  timestep                    extra
vehicle                                                     object
driver                                                      object
state    brake        2023-06-12 17:32:31.865217           float64
                      2023-06-12 17:32:31.885217           float64
                      2023-06-12 17:32:31.905217           float64
                      2023-06-12 17:32:31.925217           float64
         thrust       2023-06-12 17:32:31.865217           float64
                      2023-06-12 17:32:31.885217           float64
                      2023-06-12 17:32:31.905217           float64
                      2023-06-12 17:32:31.925217           float64
         velocity     2023-06-12 17:32:31.865217           float64
                      2023-06-12 17:32:31.885217           float64
                      2023-06-12 17:32:31.905217           float64
                      2023-06-12 17:32:31.925217           float64
action

In [96]:
dfs_episodes.columns

MultiIndex([('vehicle',         '',                         '',   ''),
            ( 'driver',         '',                         '',   ''),
            (  'state',    'brake', 2023-06-12 17:32:31.865217,   ''),
            (  'state',    'brake', 2023-06-12 17:32:31.885217,   ''),
            (  'state',    'brake', 2023-06-12 17:32:31.905217,   ''),
            (  'state',    'brake', 2023-06-12 17:32:31.925217,   ''),
            (  'state',   'thrust', 2023-06-12 17:32:31.865217,   ''),
            (  'state',   'thrust', 2023-06-12 17:32:31.885217,   ''),
            (  'state',   'thrust', 2023-06-12 17:32:31.905217,   ''),
            (  'state',   'thrust', 2023-06-12 17:32:31.925217,   ''),
            (  'state', 'velocity', 2023-06-12 17:32:31.865217,   ''),
            (  'state', 'velocity', 2023-06-12 17:32:31.885217,   ''),
            (  'state', 'velocity', 2023-06-12 17:32:31.905217,   ''),
            (  'state', 'velocity', 2023-06-12 17:32:31.925217,   ''),
      

In [97]:

dfs_episodes['vehicle'] = dfs_episodes['vehicle'].astype("category")
dfs_episodes['driver'] = dfs_episodes['driver'].astype("category")
# dfs_episodes.dtypes
# dfs_episodes.columns
# dfs_episodes[['vehicle','driver']].dtypes
# dfs_episodes[['vehicle','driver']]
# dfs_episodes[['vehicle','driver']].iloc[:3].dtypes
# sliced = dfs_episodes[['vehicle']].iloc[:3].stack(level=[-1,-2])
# type(sliced)
# sliced.index
# sliced.values

dfs_episodes_sorted = dfs_episodes.sort_index(
    axis=1, level=[0, 1, 2, 3], ascending=[True, True, True, True]
)
dfs_episodes_sorted.sort_index(axis=0, inplace=True)
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=1, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted = dfs_episodes.sort_index(axis=0, level=[0,1], ascending=[True, True])
# dfs_episodes_sorted.sort_index(axis=1,inplace=True)
dfs_episodes_sorted
dfs_episodes_sorted.columns

# dfs_episodes[['driver'], ['vehicle']] = dfs_episodes[['driver', 'vehicle']].apply(lambda x: x.cat.codes)

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                     

MultiIndex([( 'action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  0.5),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.75),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  1.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.25),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.5),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.75),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  1.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.25),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.5),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.75),
      

## dataframe slicing

In [98]:
ts
ep_start = ts - pd.Timedelta(9, 'D')
ep_start
# time_slice
episode_slice = pd.date_range(ep_start, periods=6, freq='D')
episode_slice

Timestamp('2023-06-12 17:32:31.865217')

Timestamp('2023-06-03 17:32:31.865217')

DatetimeIndex(['2023-06-03 17:32:31.865217', '2023-06-04 17:32:31.865217',
               '2023-06-05 17:32:31.865217', '2023-06-06 17:32:31.865217',
               '2023-06-07 17:32:31.865217', '2023-06-08 17:32:31.865217'],
              dtype='datetime64[ns]', freq='D')

In [99]:
timestamp_slice = pd.date_range(ts, periods=3, freq='H') + pd.Timedelta(10, 'min')
timestamp_slice

DatetimeIndex(['2023-06-12 17:42:31.865217', '2023-06-12 18:42:31.865217',
               '2023-06-12 19:42:31.865217'],
              dtype='datetime64[ns]', freq='H')

In [100]:
dfs_episodes_sorted

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                     

In [101]:
dfs_episodes_sorted.index
dfs_episodes_sorted.columns

MultiIndex([('2023-06-05 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-07 18:46:10.5755

MultiIndex([( 'action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  0.5),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.75),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  1.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.25),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.5),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.75),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  1.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.25),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.5),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.75),
      

In [102]:
# dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3]]
# dfs_selected_episodes = dfs_episodes_sorted.loc[trucks[0].vid, drivers[1].pid, episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
idx = pd.IndexSlice
dfs_selected_episodes = dfs_episodes_sorted.loc[
    idx[episode_slice[0] : episode_slice[5], timestamp_slice[0] : timestamp_slice[2]],
    idx['action':'driver'],
]
dfs_selected_episodes
# dfs_selected_episodes[['action', 'reward']]

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0   
                           2023-06-12 19:32:31.865217                       18.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0   
                           2023-06-12 19:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0  \
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   

tuple                                                                              
measurement                                                                   70   
timestep                                              2023-06-12 17:32:32.005217   
extra                                                                        0.0   
episodestart               tim

In [103]:
# dfs_selected_episodes_columns = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]][['action', 'reward']]
# dfs_selected_episodes_columns

In [104]:
# dfs_episode_all_sorted = dfs_episode_all.sort_index(axis=1)
# dfs_episode_all_sorted = dfs_episode_all_sorted.sort_index(axis=0)
# dfs_episode_all_sorted

In [105]:

# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(slice('VB6'), slice('wang-cheng'), slice(None), slice(None)),:]
# idx = pd.IndexSlice
# dfs_episode_all_sorted.loc[(idx['VB6'], idx['wang-cheng','zheng-longfei'], slice(None), slice(None)),:]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns = dfs_episodes_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns

In [106]:

ts_new = pd.to_datetime((datetime.now()))
episodestart = ts_new - pd.Timedelta(10, 'd') - pd.Timedelta(6, 'h')
episodestart
episode_slice = pd.date_range(episodestart, periods=20, freq='d') + pd.Timedelta(
    10, 'min'
)
episode_slice
ts
time_slice = pd.date_range(ts, periods=6, freq='h') - pd.Timedelta(10, 'min')
time_slice


Timestamp('2023-06-02 12:46:10.850872')

DatetimeIndex(['2023-06-02 12:56:10.850872', '2023-06-03 12:56:10.850872',
               '2023-06-04 12:56:10.850872', '2023-06-05 12:56:10.850872',
               '2023-06-06 12:56:10.850872', '2023-06-07 12:56:10.850872',
               '2023-06-08 12:56:10.850872', '2023-06-09 12:56:10.850872',
               '2023-06-10 12:56:10.850872', '2023-06-11 12:56:10.850872',
               '2023-06-12 12:56:10.850872', '2023-06-13 12:56:10.850872',
               '2023-06-14 12:56:10.850872', '2023-06-15 12:56:10.850872',
               '2023-06-16 12:56:10.850872', '2023-06-17 12:56:10.850872',
               '2023-06-18 12:56:10.850872', '2023-06-19 12:56:10.850872',
               '2023-06-20 12:56:10.850872', '2023-06-21 12:56:10.850872'],
              dtype='datetime64[ns]', freq='D')

Timestamp('2023-06-12 17:32:31.865217')

DatetimeIndex(['2023-06-12 17:22:31.865217', '2023-06-12 18:22:31.865217',
               '2023-06-12 19:22:31.865217', '2023-06-12 20:22:31.865217',
               '2023-06-12 21:22:31.865217', '2023-06-12 22:22:31.865217'],
              dtype='datetime64[ns]', freq='H')

In [107]:

episode_slice[4]: episode_slice[8]
idx[episode_slice[4] : episode_slice[8]]
idx[episode_slice[4] : episode_slice[8], slice(None)]

slice(Timestamp('2023-06-06 12:56:10.850872'), Timestamp('2023-06-10 12:56:10.850872'), None)

(slice(Timestamp('2023-06-06 12:56:10.850872'), Timestamp('2023-06-10 12:56:10.850872'), None),
 slice(None, None, None))

In [108]:
episode_slice[4]
episode_slice[11]
# dfs_selected_episodes = dfs_episodes_sorted.loc[idx['VB7'], slice(None), episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes
# epislice = idx(episode_slice[4]:episode_slice[11])

# dfs_selected_episodes = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[11],slice(None)], idx[dfs_episodes_sorted['driver']==drivers[0].pid]]
dfs_selected_episodes1 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers_by_id['wang-cheng'].pid
]
dfs_selected_episodes1
dfs_selected_episodes2 = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['driver'] == drivers_by_id['wang-cheng'].pid
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes2
dfs_selected_episodes3 = dfs_episodes_sorted.loc[
    (dfs_episodes_sorted['driver'] == drivers_by_id['wang-cheng'].pid)
    & (dfs_episodes_sorted['vehicle'] == trucks_by_name['VB7'].vid)
].loc[idx[episode_slice[4] : episode_slice[8]]]
dfs_selected_episodes3
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[idx[episode_slice[4]:episode_slice[8],slice(None)]].loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)]
# dfs_selected_episodes31 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid) & (idx[episode_slice[4]:episode_slice[8],slice(None)])]
# dfs_selected_episodes31
# dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
# dfs_selected_episodes = dfs_episodes_sorted.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11]]
# dfs_selected_episodes
# dfs_selected_episodes = dfs_episode_all.loc[slice(None), idx['zheng-longfei'], episode_slice[4]:episode_slice[11], time_slice[0]:time_slice[3]]
# dfs_selected_episodes


Timestamp('2023-06-06 12:56:10.850872')

Timestamp('2023-06-13 12:56:10.850872')

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   
                           2023-06-12 20:32:31.865217                       18.0   
                          

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   
                           2023-06-12 20:32:31.865217                       18.0   
                           2023-06-12 21:32:31.865217                       18.0   
                           2023-06-12 22:32:31.865217                       18.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0   
                           2023-06-12 19:32:31.865217                       18.0   
                           2023-06-12 20:32:31.865217                       18.0   
                           2023-06-12 21:32:31.865217                       18.0   
                           2023-06-12 22:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra              

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   
                           2023-06-12 20:32:31.865217                       18.0   
                           2023-06-12 21:32:31.865217                       18.0   
                           2023-06-12 22:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0  \
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 20:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 21:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 22:32:31.865217  19.0  20.0  21.0  22.0   

tuple                                                  ...   
measurement                                            ...   
timestep                                               ...   
extra                                                  ...   
episodestart               timestamp                   ...   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  ...  \
                           2023-06-12 19:32:31.865217  ...   
                           2023-06-12 20:32:31.865217  ...   
                           2023-06-12 21:32:31.865217  ...   
                           2023-06-12 22:32:31.865217  ...   

tuple                                                                      

In [109]:
idx[time_slice[0] : time_slice[3]]
dfs_selected_episodes3.columns

slice(Timestamp('2023-06-12 17:22:31.865217'), Timestamp('2023-06-12 20:22:31.865217'), None)

MultiIndex([( 'action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  0.5),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.75),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  1.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.25),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.5),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.75),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  1.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.25),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.5),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.75),
      

In [126]:
time_slice2 = pd.date_range(ts, periods=6, freq='S')
time_slice2
dfs_selected_episodes3_sorted = dfs_selected_episodes3.sort_index(axis=0).sort_index(axis=1)[['action']]
dfs_selected_episodes3_sorted
# dfs_selected_episodes3.loc[:, idx[:, :, 0:2, :]]
# dfs_selected_episodes3_sorted.loc[:, idx[:, :,time_slice2[0]:time_slice2[1], :]]
# dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[2]],:]
dfs_selected_episodes3.loc[idx[:, time_slice[0] : time_slice[3]], :]

DatetimeIndex(['2023-06-12 17:32:31.865217', '2023-06-12 17:32:32.865217',
               '2023-06-12 17:32:33.865217', '2023-06-12 17:32:34.865217',
               '2023-06-12 17:32:35.865217', '2023-06-12 17:32:36.865217'],
              dtype='datetime64[ns]', freq='S')

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   
                           2023-06-12 20:32:31.865217                       18.0   
                           2023-06-12 21:32:31.865217                       18.0   
                           2023-06-12 22:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0  \
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 20:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 21:32:31.865217  19.0  20.0  21.0  22.0   
                           2023-06-12 22:32:31.865217  19.0  20.0  21.0  22.0   

tuple                                                                              
measurement                                                                   70   
timestep                                              2023-06-12 17:32:32.005217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       23.0  \
                           2023-06-12 19:32:31.865217                       23.0   
                           2023-06-12 20:32:31.865217                       23.0   
                        

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0  \
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   

tuple                                                  ...   
measurement                                            ...   
timestep                                               ...   
extra                                                  ...   
episodestart               timestamp                   ...   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  ...  \
                           2023-06-12 19:32:31.865217  ...   

tuple                                                                      state   
measurement                                                                brake   
timestep                                              2023-06-12 17:32:31.925217   
extra                                                                              
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       11.0  \
                           2023-06-12 19:32:31.865217                       11.0   

tuple                                                                              
measurement                                                               thrust   
timestep                                              2023-06-12 17:32:31.865217   
extra                                                                              
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                        4.0  \
                           2023-06-12 19:32:31.865217                        4.0   

tuple                                                                    

In [127]:
dfs_selected_episodes3.index
# dfs_selected_episodes3.columns
time_slice[0]
time_slice[3]
idx[:, time_slice[0] : time_slice[3]]
dfs_selected_episodes4 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], :
]
# dfs_selected_episodes4 = dfs_selected_episodes3.loc[idx[:,time_slice[0]:time_slice[3],:],:]

dfs_selected_episodes4
dfs_selected_episodes41 = dfs_selected_episodes3.loc[
    idx[:, time_slice[0] : time_slice[3]], 'action':'driver'
]
dfs_selected_episodes41
# dfs_selected_episodes4 = dfs_episodes_sorted.loc[ (dfs_episodes_sorted['driver']==drivers[0].pid) & (dfs_episodes_sorted['vehicle']==trucks[0].vid)].loc[idx[episode_slice[4]:episode_slice[8],time_slice[0]:time_slice[3]]]
# dfs_selected_episodes4

MultiIndex([('2023-06-06 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 22:32:31.865217')],
           names=['episodestart', 'timestamp'])

Timestamp('2023-06-12 17:22:31.865217')

Timestamp('2023-06-12 20:22:31.865217')

(slice(None, None, None),
 slice(Timestamp('2023-06-12 17:22:31.865217'), Timestamp('2023-06-12 20:22:31.865217'), None))

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0  \
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   

tuple                                                  ...   
measurement                                            ...   
timestep                                               ...   
extra                                                  ...   
episodestart               timestamp                   ...   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  ...  \
                           2023-06-12 19:32:31.865217  ...   

tuple                                                                      state   
measurement                                                                brake   
timestep                                              2023-06-12 17:32:31.925217   
extra                                                                              
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       11.0  \
                           2023-06-12 19:32:31.865217                       11.0   

tuple                                                                              
measurement                                                               thrust   
timestep                                              2023-06-12 17:32:31.865217   
extra                                                                              
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                        4.0  \
                           2023-06-12 19:32:31.865217                        4.0   

tuple                                                                    

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   

tuple                                                                              
measurement                                                                   60   
timestep                                              2023-06-12 17:32:31.985217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       18.0  \
                           2023-06-12 19:32:31.865217                       18.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  19.0  20.0  21.0  22.0  \
                           2023-06-12 19:32:31.865217  19.0  20.0  21.0  22.0   

tuple                                                                              
measurement                                                                   70   
timestep                                              2023-06-12 17:32:32.005217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       23.0  \
                           2023-06-12 19:32:31.865217                       23.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  24.0  25.0  26.0  27.0  \
                           2023-06-12 19:32:31.865217  24.0  25.0  26.0  27.0   

tuple                                                      driver  
measurement                                                        
timestep                                                           
extra                                                              
episodestart               timestamp                               
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  wang-cheng  
                           2023-06-12 19:32:31.865217  wang-cheng

In [128]:

# dfs_selected_episodes = dfs_episode_all.loc['VB7', 'zheng-longfei', episode_slice[0]:episode_slice[3], timestamp_slice[0]:timestamp_slice[2]]
# dfs_selected_episodes_columns = dfs_episode_all_sorted.loc[(idx['VB7'], idx['zheng-longfei'], slice(None), slice(None)),['state','n_state']]
# dfs_selected_episodes_columns


In [129]:
dfs_episodes_sorted
# dfs_episode.loc[:,'state']
dfs_slice = dfs_episodes_sorted.loc[
    dfs_episodes_sorted['vehicle'] == 'MP73', ['vehicle', 'driver', 'state', 'reward']
]
dfs_slice

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                     

tuple                                                 vehicle         driver   
measurement                                                                    
timestep                                                                       
extra                                                                          
episodestart               timestamp                                           
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217    MP73  zheng-longfei  \
                           2023-06-12 19:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 20:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 21:32:31.865217    MP73  zheng-longfei   
                           2023-06-12 22:32:31.865217    MP73  zheng-longfei   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217    MP73     wang-cheng   
                           2023-06-12 19:32:31.865217    MP73     wang-cheng   
                           2023-06-12 20:32:31.865217    MP73     wang-cheng   
                           2023-06-12 21:32:31.865217    MP73     wang-cheng   
                           2023-06-12 22:32:31.865217    MP73     wang-cheng   

tuple                                                                      state   
measurement                                                                brake   
timestep                                              2023-06-12 17:32:31.865217   
extra                                                                              
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0  \
                           2023-06-12 19:32:31.865217                        8.0   
                           2023-06-12 20:32:31.865217                        8.0   
                           2023-06-12 21:32:31.865217                        8.0   
                           2023-06-12 22:32:31.865217                        8.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                        8.0   
                           2023-06-12 19:32:31.865217                        8.0   
                           2023-06-12 20:32:31.865217                        8.0   
                           2023-06-12 21:32:31.865217                        8.0   
                           2023-06-12 22:32:31.865217                        8.0   

tuple                                                                              
measurement                                                                        
timestep                                              2023-06-12 17:32:31.885217   
extra                                                                              
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                        9.0  \
                           2023-06-12 19:32:31.865217                        9.0   
                           2023-06-12 20:32:31.865217                        9.0   
                           2023-06-12 21:32:31.865217                        9.0   
                           2023-06-12 22:32:31.865217                        9.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                        9.0   
                           2023-06-12 19:32:31.865217                        9.0   
                           2023-06-12 20:32:31.865217                        9.0   
                           2023-06-12 21:32:31.865217                        9.0   
                           2023-06-12 22:32:31.865217                        9.0   

tuple                                                                              
measurement                                                                        
timestep                                              2023-06-12 17:32:31.905217   
extra                    

# Saved episodes as parquet in multi-level folders "vehicle/driver/episode.parquet"

In [130]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from datetime import datetime

import pyarrow as pa
import pyarrow.parquet as pq
import platform

print('Python: ', platform.python_version())
print('pandas: ', pd.__version__)
print('pyarrow: ', pa.__version__)


Python:  3.10.9
pandas:  2.0.1
pyarrow:  11.0.0


In [131]:
os.chdir('../../data')

In [132]:

os.getcwd()

'/home/n/devel/anting_remote/eos/data'

## create parquet files

In [133]:
dfs_episodes_sorted
dfs_episodes_sorted.index
dfs_episodes_sorted.columns
dfs_episodes_sorted.dtypes

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                     

MultiIndex([('2023-06-05 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-07 18:46:10.5755

MultiIndex([( 'action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  0.5),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.75),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  1.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.25),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.5),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.75),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  1.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.25),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.5),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.75),
      

tuple    measurement  timestep                    extra
action   50           2023-06-12 17:32:31.965217  0.0       float64
                                                  0.25      float64
                                                  0.5       float64
                                                  0.75      float64
                                                  1.0       float64
         60           2023-06-12 17:32:31.985217  0.0       float64
                                                  0.25      float64
                                                  0.5       float64
                                                  0.75      float64
                                                  1.0       float64
         70           2023-06-12 17:32:32.005217  0.0       float64
                                                  0.25      float64
                                                  0.5       float64
                                                  0.75      

In [134]:
# dfs_episode4dask = dfs_episode_all_sorted.swaplevel(0,-1, axis=0).swaplevel(1,3, axis=0).swaplevel(2,3, axis=0).unstack()
# dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# dfs_episode4dask
# # dfs_episode4dask = dfs_episode4dask.swaplevel(0,-1, axis=1).swaplevel(-2,-1, axis=1)
# # dfs_episode4dask.index
# # dfs_episode4dask.index
# # dfs_episode4dask.columns
# # dfs_episode4dask
# # dfs_episode_all_stack = dfs_episode_all.unstack(level=[-1,-2,-3,])
# # dfs_episode_all_stack

In [135]:
# dfs_episode4dask1 = dfs_episode4dask.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1)
# # dfs_episode4dask1
# dfs_episode4dask2 = dfs_episode4dask1.unstack().swaplevel(-2,-1, axis=1).swaplevel(-2,-3,axis=1).swaplevel(-3,-4,axis=1).swaplevel(-4,-5,axis=1)
# dfs_episode4dask2.sort_index(axis=1, inplace=True)
# dfs_episode4dask2


### decoding multiindex dataframe to flat index for Dask datframe

In [156]:
dfs_episodes_sorted.columns
cols = dfs_episodes_sorted.columns.to_flat_index()
cols

## 3 level index
# cols_str = [
#     f'{x[0]}_{x[1]}_{x[2]}'
#     if (x[1] != '')
#     else f'{x[0]}__{x[2]}'
#     if (x[2] != '')
#     else f'{x[0]}'
#     for x in cols
# ]

## 4 level index
cols_str = [
    f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[3]}'
    if (x[1] != '' and x[3] != '')
    else f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[1] != '' and x[3] == '')
    else f'{x[0]}__{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[2] != '')
    else f'{x[0]}___'
    for x in cols
]
cols_str

MultiIndex([( 'action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.25),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  0.5),
            ( 'action',         50, 2023-06-12 17:32:31.965217, 0.75),
            ( 'action',         50, 2023-06-12 17:32:31.965217,  1.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.0),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.25),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  0.5),
            ( 'action',         60, 2023-06-12 17:32:31.985217, 0.75),
            ( 'action',         60, 2023-06-12 17:32:31.985217,  1.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.0),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.25),
            ( 'action',         70, 2023-06-12 17:32:32.005217,  0.5),
            ( 'action',         70, 2023-06-12 17:32:32.005217, 0.75),
      

Index([       ('action', 50, 2023-06-12 17:32:31.965217, 0.0),
             ('action', 50, 2023-06-12 17:32:31.965217, 0.25),
              ('action', 50, 2023-06-12 17:32:31.965217, 0.5),
             ('action', 50, 2023-06-12 17:32:31.965217, 0.75),
              ('action', 50, 2023-06-12 17:32:31.965217, 1.0),
              ('action', 60, 2023-06-12 17:32:31.985217, 0.0),
             ('action', 60, 2023-06-12 17:32:31.985217, 0.25),
              ('action', 60, 2023-06-12 17:32:31.985217, 0.5),
             ('action', 60, 2023-06-12 17:32:31.985217, 0.75),
              ('action', 60, 2023-06-12 17:32:31.985217, 1.0),
              ('action', 70, 2023-06-12 17:32:32.005217, 0.0),
             ('action', 70, 2023-06-12 17:32:32.005217, 0.25),
              ('action', 70, 2023-06-12 17:32:32.005217, 0.5),
             ('action', 70, 2023-06-12 17:32:32.005217, 0.75),
              ('action', 70, 2023-06-12 17:32:32.005217, 1.0),
                                       ('driver', '', '

['action_50_2023-06-12T17:32:31.965217_0.0',
 'action_50_2023-06-12T17:32:31.965217_0.25',
 'action_50_2023-06-12T17:32:31.965217_0.5',
 'action_50_2023-06-12T17:32:31.965217_0.75',
 'action_50_2023-06-12T17:32:31.965217_1.0',
 'action_60_2023-06-12T17:32:31.985217_0.0',
 'action_60_2023-06-12T17:32:31.985217_0.25',
 'action_60_2023-06-12T17:32:31.985217_0.5',
 'action_60_2023-06-12T17:32:31.985217_0.75',
 'action_60_2023-06-12T17:32:31.985217_1.0',
 'action_70_2023-06-12T17:32:32.005217_0.0',
 'action_70_2023-06-12T17:32:32.005217_0.25',
 'action_70_2023-06-12T17:32:32.005217_0.5',
 'action_70_2023-06-12T17:32:32.005217_0.75',
 'action_70_2023-06-12T17:32:32.005217_1.0',
 'driver___',
 'nstate_brake_2023-06-12T17:32:36.865217_',
 'nstate_brake_2023-06-12T17:32:36.885217_',
 'nstate_brake_2023-06-12T17:32:36.905217_',
 'nstate_brake_2023-06-12T17:32:36.925217_',
 'nstate_thrust_2023-06-12T17:32:36.865217_',
 'nstate_thrust_2023-06-12T17:32:36.885217_',
 'nstate_thrust_2023-06-12T17:32:

In [190]:
# encoding index and columns
dfs_episodes_sorted_flat_col = dfs_episodes_sorted.copy()
cols = dfs_episodes_sorted.columns.to_flat_index()
dfs_episodes_sorted_flat_col.columns = [
    f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[3]}'
    if (x[1] != '' and x[3] != '')
    else f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[1] != '' and x[3] == '')
    else f'{x[0]}__{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[2] != '')
    else f'{x[0]}___'
    for x in cols
]
dfs_episodes_sorted_flat_ind = dfs_episodes_sorted.copy()
idx = dfs_episodes_sorted.index.to_flat_index()
dfs_episodes_sorted_flat_ind.index = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}' if x[1] != '' else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}' for x in idx
]
# dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat = dfs_episodes_sorted.copy()
dfs_episodes_sorted_flat.columns = [
    f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[3]}'
    if (x[1] != '' and x[3] != '')
    else f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[1] != '' and x[3] == '')
    else f'{x[0]}__{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[2] != '')
    else f'{x[0]}___'
    for x in cols

]
dfs_episodes_sorted_flat.index = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}' if x[1] != '' else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}' for x in idx
]
dfs_episodes_sorted_flat



,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,...,state_brake_2023-06-12T17:32:31.925217_,state_thrust_2023-06-12T17:32:31.865217_,state_thrust_2023-06-12T17:32:31.885217_,state_thrust_2023-06-12T17:32:31.905217_,state_thrust_2023-06-12T17:32:31.925217_,state_velocity_2023-06-12T17:32:31.865217_,state_velocity_2023-06-12T17:32:31.885217_,state_velocity_2023-06-12T17:32:31.905217_,state_velocity_2023-06-12T17:32:31.925217_,vehicle___
2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-06T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7


In [158]:
dfs_episodes_sorted_flat_ind
dfs_episodes_sorted_flat_col
dfs_episodes_sorted_flat
dfs_episodes_sorted_flat.dtypes

tuple                                                                  action   
measurement                                                                50   
timestep                                           2023-06-12 17:32:31.965217   
extra                                                                     0.0   
2023-06-05T18:46:10.575552_2023-06-12T18:32:31....                       13.0  \
2023-06-05T18:46:10.575552_2023-06-12T19:32:31....                       13.0   
2023-06-05T18:46:10.575552_2023-06-12T20:32:31....                       13.0   
2023-06-05T18:46:10.575552_2023-06-12T21:32:31....                       13.0   
2023-06-05T18:46:10.575552_2023-06-12T22:32:31....                       13.0   
2023-06-06T18:46:10.575552_2023-06-12T18:32:31....                       13.0   
2023-06-06T18:46:10.575552_2023-06-12T19:32:31....                       13.0   
2023-06-06T18:46:10.575552_2023-06-12T20:32:31....                       13.0   
2023-06-06T18:46:10.575552_2023-06-12T21:32:31....                       13.0   
2023-06-06T18:46:10.575552_2023-06-12T22:32:31....                       13.0   
2023-06-07T18:46:10.575552_2023-06-12T18:32:31....                       13.0   
2023-06-07T18:46:10.575552_2023-06-12T19:32:31....                       13.0   
2023-06-07T18:46:10.575552_2023-06-12T20:32:31....                       13.0   
2023-06-07T18:46:10.575552_2023-06-12T21:32:31....                       13.0   
2023-06-07T18:46:10.575552_2023-06-12T22:32:31....                       13.0   
2023-06-10T18:46:10.575552_2023-06-12T18:32:31....                       13.0   
2023-06-10T18:46:10.575552_2023-06-12T19:32:31....                       13.0   
2023-06-10T18:46:10.575552_2023-06-12T20:32:31....                       13.0   
2023-06-10T18:46:10.575552_2023-06-12T21:32:31....                       13.0   
2023-06-10T18:46:10.575552_2023-06-12T22:32:31....                       13.0   
2023-06-12T17:32:31.865217_2023-06-12T18:32:31....                       13.0   
2023-06-12T17:32:31.865217_2023-06-12T19:32:31....                       13.0   
2023-06-12T17:32:31.865217_2023-06-12T20:32:31....                       13.0   
2023-06-12T17:32:31.865217_2023-06-12T21:32:31....                       13.0   
2023-06-12T17:32:31.865217_2023-06-12T22:32:31....                       13.0   

tuple                                                                        
measurement                                                                  
timestep                                                                     
extra                                               0.25   0.5  0.75   1.0   
2023-06-05T18:46:10.575552_2023-06-12T18:32:31....  14.0  15.0  16.0  17.0  \
2023-06-05T18:46:10.575552_2023-06-12T19:32:31....  14.0  15.0  16.0  17.0   
2023-06-05T18:46:10.575552_2023-06-12T20:32:31....  14.0  15.0  16.0  17.0   
2023-06-05T18:46:10.575552_2023-06-12T21:32:31....  14.0  15.0  16.0  17.0   
2023-06-05T18:46:10.575552_2023-06-12T22:32:31....  14.0  15.0  16.0  17.0   
2023-06-06T18:46:10.575552_2023-06-12T18:32:31....  14.0  15.0  16.0  17.0   
2023-06-06T18:46:10.575552_2023-06-12T19:32:31....  14.0  15.0  16.0  17.0   
2023-06-06T18:46:10.575552_2023-06-12T20:32:31....  14.0  15.0  16.0  17.0   
2023-06-06T18:46:10.575552_2023-06-12T21:32:31....  14.0  15.0  16.0  17.0   
2023-06-06T18:46:10.575552_2023-06-12T22:32:31....  14.0  15.0  16.0  17.0   
2023-06-07T18:46:10.575552_2023-06-12T18:32:31....  14.0  15.0  16.0  17.0   
2023-06-07T18:46:10.575552_2023-06-12T19:32:31....  14.0  15.0  16.0  17.0   
2023-06-07T18:46:10.575552_2023-06-12T20:32:31....  14.0  15.0  16.0  17.0   
2023-06-07T18:46:10.575552_2023-06-12T21:32:31....  14.0  15.0  16.0  17.0   
2023-06-07T18:46:10.575552_2023-06-12T22:32:31....  14.0  15.0  16.0  17.0   
2023-06-10T18:46:10.575552_2023-06-12T18:32:31....  14.0  15.0  16.0  17.0   
2023-06-10T18:46:10.575552_2023-06-12T19:32:31....  14.0  15.0  16.0  17.0   
2023-06-10T1

action_50_2023-06-12T17:32:31.965217_0.0   
episodestart               timestamp                                                              
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                                      13.0  \
                           2023-06-12 19:32:31.865217                                      13.0   
                           2023-06-12 20:32:31.865217                                      13.0   
                           2023-06-12 21:32:31.865217                                      13.0   
                           2023-06-12 22:32:31.865217                                      13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                                      13.0   
                           2023-06-12 19:32:31.865217                                      13.0   
                           2023-06-12 20:32:31.865217                                      13.0   
                           2023-06-12 21:32:31.865217                                      13.0   
                           2023-06-12 22:32:31.865217                                      13.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                                      13.0   
                           2023-06-12 19:32:31.865217                                      13.0   
                           2023-06-12 20:32:31.865217                                      13.0   
                           2023-06-12 21:32:31.865217                                      13.0   
                           2023-06-12 22:32:31.865217                                      13.0   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                                      13.0   
                           2023-06-12 19:32:31.865217                                      13.0   
                           2023-06-12 20:32:31.865217                                      13.0   
                           2023-06-12 21:32:31.865217                                      13.0   
                           2023-06-12 22:32:31.865217                                      13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                                      13.0   
                           2023-06-12 19:32:31.865217                                      13.0   
                           2023-06-12 20:32:31.865217                                      13.0   
                           2023-06-12 21:32:31.865217                                      13.0   
                           2023-06-12 22:32:31.865217                                      13.0   

                                                       action_50_2023-06-12T17:32:31.965217_0.25   
episodestart               timestamp                                                               
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                                       14.0  \
                           2023-06-12 19:32:31.865217                                       14.0   
                           2023-06-12 20:32:31.865217                                       14.0   
                           2023-06-12 21:32:31.865217                                       14.0   
                           2023-06-12 22:32:31.865217                                       14.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                                       14.0   
                           2023-06-12 19:32:31.865217                                       14.0   
                           2023-06-12 20:32:31.865217                                       14.0   
                           2023-06-12 21:32:31.865217                                       14.0   
                           2023-06-12 22:32:31.865217                                       14.0   
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                                       14.0   
                           2023-06-12 19:32:31.865217                            

,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,...,state_brake_2023-06-12T17:32:31.925217_,state_thrust_2023-06-12T17:32:31.865217_,state_thrust_2023-06-12T17:32:31.885217_,state_thrust_2023-06-12T17:32:31.905217_,state_thrust_2023-06-12T17:32:31.925217_,state_velocity_2023-06-12T17:32:31.865217_,state_velocity_2023-06-12T17:32:31.885217_,state_velocity_2023-06-12T17:32:31.905217_,state_velocity_2023-06-12T17:32:31.925217_,vehicle___
2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-05T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73
2023-06-06T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7
2023-06-06T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,11.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,VB7


action_50_2023-06-12T17:32:31.965217_0.0        float64
action_50_2023-06-12T17:32:31.965217_0.25       float64
action_50_2023-06-12T17:32:31.965217_0.5        float64
action_50_2023-06-12T17:32:31.965217_0.75       float64
action_50_2023-06-12T17:32:31.965217_1.0        float64
action_60_2023-06-12T17:32:31.985217_0.0        float64
action_60_2023-06-12T17:32:31.985217_0.25       float64
action_60_2023-06-12T17:32:31.985217_0.5        float64
action_60_2023-06-12T17:32:31.985217_0.75       float64
action_60_2023-06-12T17:32:31.985217_1.0        float64
action_70_2023-06-12T17:32:32.005217_0.0        float64
action_70_2023-06-12T17:32:32.005217_0.25       float64
action_70_2023-06-12T17:32:32.005217_0.5        float64
action_70_2023-06-12T17:32:32.005217_0.75       float64
action_70_2023-06-12T17:32:32.005217_1.0        float64
driver___                                      category
nstate_brake_2023-06-12T17:32:36.865217_        float64
nstate_brake_2023-06-12T17:32:36.885217_        

In [160]:
dfs_episodes_sorted_flat['vehicle___'].iloc[0:2].values.dtype

CategoricalDtype(categories=['MP73', 'VB7'], ordered=False)

In [161]:
try:
    ddf_episodes = dd.from_pandas(dfs_episodes_sorted_flat, npartitions=1)
except Exception as e:
    print(e)

try:
    ddf_episodes.index
    ddf_episodes.columns
except Exception as e:
    print(e)

ddf_episodes

Dask Index Structure:
npartitions=1
2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217    object
2023-06-12T17:32:31.865217_2023-06-12T22:32:31.865217       ...
dtype: object
Dask Name: from_pandas-index, 2 graph layers

Index(['action_50_2023-06-12T17:32:31.965217_0.0',
       'action_50_2023-06-12T17:32:31.965217_0.25',
       'action_50_2023-06-12T17:32:31.965217_0.5',
       'action_50_2023-06-12T17:32:31.965217_0.75',
       'action_50_2023-06-12T17:32:31.965217_1.0',
       'action_60_2023-06-12T17:32:31.985217_0.0',
       'action_60_2023-06-12T17:32:31.985217_0.25',
       'action_60_2023-06-12T17:32:31.985217_0.5',
       'action_60_2023-06-12T17:32:31.985217_0.75',
       'action_60_2023-06-12T17:32:31.985217_1.0',
       'action_70_2023-06-12T17:32:32.005217_0.0',
       'action_70_2023-06-12T17:32:32.005217_0.25',
       'action_70_2023-06-12T17:32:32.005217_0.5',
       'action_70_2023-06-12T17:32:32.005217_0.75',
       'action_70_2023-06-12T17:32:32.005217_1.0', 'driver___',
       'nstate_brake_2023-06-12T17:32:36.865217_',
       'nstate_brake_2023-06-12T17:32:36.885217_',
       'nstate_brake_2023-06-12T17:32:36.905217_',
       'nstate_brake_2023-06-12T17:32:36.925217_',
       'nsta

,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,action_70_2023-06-12T17:32:32.005217_0.0,action_70_2023-06-12T17:32:32.005217_0.25,action_70_2023-06-12T17:32:32.005217_0.5,action_70_2023-06-12T17:32:32.005217_0.75,action_70_2023-06-12T17:32:32.005217_1.0,driver___,nstate_brake_2023-06-12T17:32:36.865217_,nstate_brake_2023-06-12T17:32:36.885217_,nstate_brake_2023-06-12T17:32:36.905217_,nstate_brake_2023-06-12T17:32:36.925217_,nstate_thrust_2023-06-12T17:32:36.865217_,nstate_thrust_2023-06-12T17:32:36.885217_,nstate_thrust_2023-06-12T17:32:36.905217_,nstate_thrust_2023-06-12T17:32:36.925217_,nstate_velocity_2023-06-12T17:32:36.865217_,nstate_velocity_2023-06-12T17:32:36.885217_,nstate_velocity_2023-06-12T17:32:36.905217_,nstate_velocity_2023-06-12T17:32:36.925217_,reward__2023-06-12T17:32:36.865217_,state_brake_2023-06-12T17:32:31.865217_,state_brake_2023-06-12T17:32:31.885217_,state_brake_2023-06-12T17:32:31.905217_,state_brake_2023-06-12T17:32:31.925217_,state_thrust_2023-06-12T17:32:31.865217_,state_thrust_2023-06-12T17:32:31.885217_,state_thrust_2023-06-12T17:32:31.905217_,state_thrust_2023-06-12T17:32:31.925217_,state_velocity_2023-06-12T17:32:31.865217_,state_velocity_2023-06-12T17:32:31.885217_,state_velocity_2023-06-12T17:32:31.905217_,state_velocity_2023-06-12T17:32:31.925217_,vehicle___
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,category[known],float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,category[known]
2023-06-12T17:32:31.865217_2023-06-12T22:32:31.865217,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [149]:

custom_metadata = {
    b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}
custom_metadata

{b'eos': b'{"length":[3, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'}

In [163]:
try:
    with ProgressBar():
        ddf_episodes.to_parquet(
            'eos_dask_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle___', 'driver___'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

[########################################] | 100% Completed | 101.21 ms


### Read saved parquet file

In [224]:
ddf = dd.read_parquet(
    'eos_dask_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.head(5)
ddf.dtypes
ddf.npartitions
ddf.divisions
ddf._meta

,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,...,state_thrust_2023-06-12T17:32:31.865217_,state_thrust_2023-06-12T17:32:31.885217_,state_thrust_2023-06-12T17:32:31.905217_,state_thrust_2023-06-12T17:32:31.925217_,state_velocity_2023-06-12T17:32:31.865217_,state_velocity_2023-06-12T17:32:31.885217_,state_velocity_2023-06-12T17:32:31.905217_,state_velocity_2023-06-12T17:32:31.925217_,vehicle___,driver___
2023-06-07T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng


action_50_2023-06-12T17:32:31.965217_0.0        float64
action_50_2023-06-12T17:32:31.965217_0.25       float64
action_50_2023-06-12T17:32:31.965217_0.5        float64
action_50_2023-06-12T17:32:31.965217_0.75       float64
action_50_2023-06-12T17:32:31.965217_1.0        float64
action_60_2023-06-12T17:32:31.985217_0.0        float64
action_60_2023-06-12T17:32:31.985217_0.25       float64
action_60_2023-06-12T17:32:31.985217_0.5        float64
action_60_2023-06-12T17:32:31.985217_0.75       float64
action_60_2023-06-12T17:32:31.985217_1.0        float64
action_70_2023-06-12T17:32:32.005217_0.0        float64
action_70_2023-06-12T17:32:32.005217_0.25       float64
action_70_2023-06-12T17:32:32.005217_0.5        float64
action_70_2023-06-12T17:32:32.005217_0.75       float64
action_70_2023-06-12T17:32:32.005217_1.0        float64
nstate_brake_2023-06-12T17:32:36.865217_        float64
nstate_brake_2023-06-12T17:32:36.885217_        float64
nstate_brake_2023-06-12T17:32:36.905217_        

4

(None, None, None, None, None)

,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,...,state_thrust_2023-06-12T17:32:31.865217_,state_thrust_2023-06-12T17:32:31.885217_,state_thrust_2023-06-12T17:32:31.905217_,state_thrust_2023-06-12T17:32:31.925217_,state_velocity_2023-06-12T17:32:31.865217_,state_velocity_2023-06-12T17:32:31.885217_,state_velocity_2023-06-12T17:32:31.905217_,state_velocity_2023-06-12T17:32:31.925217_,vehicle___,driver___


### Get customized metadata with pyarrow

In [225]:

import pyarrow as pa
import pyarrow.parquet as pq
import json

# print('pyarrow: ', pa.__version__)

# table = pq.read_table('ddf_episodes')
# table_meta = table.schema.metadata[b'eos']
# print(table_meta)
# # custom_metadata_info = json.loads(table.schema.metadata[b'eos'])
# # print(custom_metadata_info)
# try:
#     custom_meta_info = json.loads(table_meta)
# except Exception as e:
#     print(f'Exception: {e}')

# print(custom_meta_info)

# # Print formatted output of the dichtionary custom_meta_info:
# for key, val in custom_meta_info.items():
#     print('{:15}: {}'.format(key, val))

# ALTERNATE:

table_meta = pq.read_metadata('eos_dask_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = table_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

# table_meta = pq.read_metadata('ddf_episodes/_metadata')
# # print(table_meta)
# # print(table_meta.metadata[b'eos'])
# custom_meta_info = table_meta.metadata[b'eos']
# custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

length: [3, 600, 5]
timezeone: sh
units: ['kph', 'pct', 'pct']
data_propensity: [50, 1, 4]
Dataset Description: MP vehicle from TBox


### encoding flat_index of dask dataframe to multiindex

In [226]:
df_episodes_read = ddf.compute()
df_episodes_read

,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,...,state_thrust_2023-06-12T17:32:31.865217_,state_thrust_2023-06-12T17:32:31.885217_,state_thrust_2023-06-12T17:32:31.905217_,state_thrust_2023-06-12T17:32:31.925217_,state_velocity_2023-06-12T17:32:31.865217_,state_velocity_2023-06-12T17:32:31.885217_,state_velocity_2023-06-12T17:32:31.905217_,state_velocity_2023-06-12T17:32:31.925217_,vehicle___,driver___
2023-06-07T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-07T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,wang-cheng
2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-05T18:46:10.575552_2023-06-12T19:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-05T18:46:10.575552_2023-06-12T20:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-05T18:46:10.575552_2023-06-12T21:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,zheng-longfei
2023-06-05T18:46:10.575552_2023-06-12T22:32:31.865217,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,...,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,MP73,zheng-longfei


In [227]:
# decoding
multi_cols = [tuple(col.split('_')) for col in df_episodes_read.columns]
multi_cols
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_cols)
# multi_idx
multi_idx.names = ['tuple', 'measurement', 'timestep', 'extra']
driver_vehicle = [idx for idx in multi_idx if idx[0] == 'driver' or idx[0] == 'vehicle']
driver_vehicle
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_idx

[('action', '50', '2023-06-12T17:32:31.965217', '0.0'),
 ('action', '50', '2023-06-12T17:32:31.965217', '0.25'),
 ('action', '50', '2023-06-12T17:32:31.965217', '0.5'),
 ('action', '50', '2023-06-12T17:32:31.965217', '0.75'),
 ('action', '50', '2023-06-12T17:32:31.965217', '1.0'),
 ('action', '60', '2023-06-12T17:32:31.985217', '0.0'),
 ('action', '60', '2023-06-12T17:32:31.985217', '0.25'),
 ('action', '60', '2023-06-12T17:32:31.985217', '0.5'),
 ('action', '60', '2023-06-12T17:32:31.985217', '0.75'),
 ('action', '60', '2023-06-12T17:32:31.985217', '1.0'),
 ('action', '70', '2023-06-12T17:32:32.005217', '0.0'),
 ('action', '70', '2023-06-12T17:32:32.005217', '0.25'),
 ('action', '70', '2023-06-12T17:32:32.005217', '0.5'),
 ('action', '70', '2023-06-12T17:32:32.005217', '0.75'),
 ('action', '70', '2023-06-12T17:32:32.005217', '1.0'),
 ('nstate', 'brake', '2023-06-12T17:32:36.865217', ''),
 ('nstate', 'brake', '2023-06-12T17:32:36.885217', ''),
 ('nstate', 'brake', '2023-06-12T17:32:36.

[('vehicle', '', '', ''), ('driver', '', '', '')]

In [ ]:
# pd.Series([1, np.nan,2,None])
# pd.Series([1,np.nan,2,None,pd.NA])

In [228]:
# decoding columns
from datetime import datetime
multi_tpl = [tuple(col.split('_')) for col in df_episodes_read.columns]
# multi_tpl
# multi_cols = [(col[0], int(col[1])) if len(col)==2 else col for col in multi_cols]
# multi_cols
df_episodes_read_multicol = df_episodes_read.copy()
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_idx
# multi_col
i1 = multi_col.get_level_values(0)
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i1
]  # convert index of level 2 type to int and '' if NA
i1
# i2 = multi_col.get_level_values(1).fillna('') # must be null string instead of the default pd.NA or np.nan
# i2 = [idx if isinstance(idx, int) else '' for idx in i2]
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i2
# i2 = [idx if isinstance(idx, str) else '' for idx in i2]
# i2.astype('int')
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else datetime.strptime(idx,'%Y-%m-%dT%H:%M:%S.%f') for idx in i3
]  # convert index of level 2 type to int and '' if NA
i3
i4 = multi_col.get_level_values(
    3
)  # must be null string instead of the default pd.NA or np.nan
i4 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i4
]  # convert index of level 2 type to int and '' if NA
i4

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3, i4])
multi_col.names = ['tuple', 'measurement', 'timestep', 'extra']
multi_col


# i2[-1]
# i2.dropna()
# i2
# l2 = ['' if i==nan else i  for i in i2]
# l2
# i2= i2[:-2].astype('int')
# i2 = i2.append(pd.Index(['','']))
# l2 = [''] + list(l2.values[:-1])
# print(l2)
# arrays = [ for i2 in l2 for i1 in l1]
# l2 = l2[:-2].astype('int') + ['', '']
# multi_col.set_levels(ll, level=['tuple','step'])
# driver_vehicle = [idx for idx in multi_idx if idx[0]=='driver' or idx[0]=='vehicle']
# multi_tpl =  [(col[0],)  if col[0]=='driver' or col[0]=='vehicle' else (col[0],int(col[1])) for col in multi_col]
# multi_idx1 = pd.MultiIndex.from_tuples(multi_idx1)
# multi_idx = multi_idx.set_levels([multi_idx.levels[0], multi_idx.levels[1].astype(int)])
# multi_col = pd.MultiIndex.from_tuples(multi_tpl)
# multi_col

['action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'action',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'nstate',
 'reward',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'state',
 'vehicle',
 'driver']

['50',
 '50',
 '50',
 '50',
 '50',
 '60',
 '60',
 '60',
 '60',
 '60',
 '70',
 '70',
 '70',
 '70',
 '70',
 'brake',
 'brake',
 'brake',
 'brake',
 'thrust',
 'thrust',
 'thrust',
 'thrust',
 'velocity',
 'velocity',
 'velocity',
 'velocity',
 '',
 'brake',
 'brake',
 'brake',
 'brake',
 'thrust',
 'thrust',
 'thrust',
 'thrust',
 'velocity',
 'velocity',
 'velocity',
 'velocity',
 '',
 '']

[datetime.datetime(2023, 6, 12, 17, 32, 31, 965217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 965217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 965217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 965217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 965217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 985217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 985217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 985217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 985217),
 datetime.datetime(2023, 6, 12, 17, 32, 31, 985217),
 datetime.datetime(2023, 6, 12, 17, 32, 32, 5217),
 datetime.datetime(2023, 6, 12, 17, 32, 32, 5217),
 datetime.datetime(2023, 6, 12, 17, 32, 32, 5217),
 datetime.datetime(2023, 6, 12, 17, 32, 32, 5217),
 datetime.datetime(2023, 6, 12, 17, 32, 32, 5217),
 datetime.datetime(2023, 6, 12, 17, 32, 36, 865217),
 datetime.datetime(2023, 6, 12, 17, 32, 36, 885217),
 datetime.datetime(2023, 6, 12, 17, 32, 36, 905217),
 datetime.datetime(2023, 6, 12, 17, 32, 36, 925217),
 da

['0.0',
 '0.25',
 '0.5',
 '0.75',
 '1.0',
 '0.0',
 '0.25',
 '0.5',
 '0.75',
 '1.0',
 '0.0',
 '0.25',
 '0.5',
 '0.75',
 '1.0',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

MultiIndex([( 'action',       '50', 2023-06-12 17:32:31.965217,  '0.0'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217, '0.25'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217,  '0.5'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217, '0.75'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217,  '1.0'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217,  '0.0'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217, '0.25'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217,  '0.5'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217, '0.75'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217,  '1.0'),
            ( 'action',       '70', 2023-06-12 17:32:32.005217,  '0.0'),
            ( 'action',       '70', 2023-06-12 17:32:32.005217, '0.25'),
            ( 'action',       '70', 2023-06-12 17:32:32.005217,  '0.5'),
            ( 'action',       '70', 2023-06-12 17:3

In [229]:
# decoding index
multi_tuple = [tuple(idx.split('_')) for idx in df_episodes_read.index]
multi_idx = pd.MultiIndex.from_tuples(multi_tuple)
# multi_idx
i0 = multi_idx.get_level_values(
    0
)  # must be null string instead of the default pd.NA or np.nan
i0 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else datetime.strptime(idx,'%Y-%m-%dT%H:%M:%S.%f') for idx in i0
]  # convert index of level 2 type to int and '' if NA
i1 = multi_idx.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i1 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else datetime.strptime(idx,'%Y-%m-%dT%H:%M:%S.%f') for idx in i1
]  # convert index of level 2 type to int and '' if NA
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_arrays([i0,i1],names=('episodestart', 'timestamp'))

multi_idx.dtypes
multi_idx
len(multi_idx)

episodestart    datetime64[ns]
timestamp       datetime64[ns]
dtype: object

MultiIndex([('2023-06-07 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-06 18:46:10.5755

25

In [230]:
df_episodes_read_multi = df_episodes_read.copy()
df_episodes_read_multi.columns = multi_col
df_episodes_read_multi.index = multi_idx
df_episodes_read_multi
df_episodes_read_multi.columns
df_episodes_read_multi.index
df_episodes_read_multi.dtypes

tuple                                                                     action   
measurement                                                                   50   
timestep                                              2023-06-12 17:32:31.965217   
extra                                                                        0.0   
episodestart               timestamp                                               
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   
2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                           2023-06-12 19:32:31.865217                       13.0   
                           2023-06-12 20:32:31.865217                       13.0   
                           2023-06-12 21:32:31.865217                       13.0   
                           2023-06-12 22:32:31.865217                       13.0   

tuple                                                                           
measurement                                                                     
timestep                                                                        
extra                                                  0.25   0.5  0.75   1.0   
episodestart               timestamp                                            
2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0  \
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 21:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 22:32:31.865217  14.0  15.0  16.0  17.0   
2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 19:32:31.865217  14.0  15.0  16.0  17.0   
                           2023-06-12 20:32:31.865217  14.0  15.0  16.0  17.0   
                     

MultiIndex([( 'action',       '50', 2023-06-12 17:32:31.965217,  '0.0'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217, '0.25'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217,  '0.5'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217, '0.75'),
            ( 'action',       '50', 2023-06-12 17:32:31.965217,  '1.0'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217,  '0.0'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217, '0.25'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217,  '0.5'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217, '0.75'),
            ( 'action',       '60', 2023-06-12 17:32:31.985217,  '1.0'),
            ( 'action',       '70', 2023-06-12 17:32:32.005217,  '0.0'),
            ( 'action',       '70', 2023-06-12 17:32:32.005217, '0.25'),
            ( 'action',       '70', 2023-06-12 17:32:32.005217,  '0.5'),
            ( 'action',       '70', 2023-06-12 17:3

MultiIndex([('2023-06-07 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-06 18:46:10.5755

tuple    measurement  timestep                    extra
action   50           2023-06-12 17:32:31.965217  0.0       float64
                                                  0.25      float64
                                                  0.5       float64
                                                  0.75      float64
                                                  1.0       float64
         60           2023-06-12 17:32:31.985217  0.0       float64
                                                  0.25      float64
                                                  0.5       float64
                                                  0.75      float64
                                                  1.0       float64
         70           2023-06-12 17:32:32.005217  0.0       float64
                                                  0.25      float64
                                                  0.5       float64
                                                  0.75      

### recover deep multiindex from columns ['vehicle','driver']

In [231]:
df_episodes_read_multi_setindex = df_episodes_read_multi.set_index(
    ['vehicle', 'driver', df_episodes_read_multi.index]
)
df_episodes_read_multi_setindex.index
df_episodes_read_multi_setindex.columns
df_episodes_read_multi_setindex

MultiIndex([('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73',    'wang-cheng', '2023-06-07 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ('MP73', 'zheng-longfei', '2023-06-05 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', '2023-06-06 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', '2023-06-06 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', '2023-06-06 18:46:10.575552', ...),
            ( 'VB7',    'wang-cheng', 

MultiIndex([('action',       '50', 2023-06-12 17:32:31.965217,  '0.0'),
            ('action',       '50', 2023-06-12 17:32:31.965217, '0.25'),
            ('action',       '50', 2023-06-12 17:32:31.965217,  '0.5'),
            ('action',       '50', 2023-06-12 17:32:31.965217, '0.75'),
            ('action',       '50', 2023-06-12 17:32:31.965217,  '1.0'),
            ('action',       '60', 2023-06-12 17:32:31.985217,  '0.0'),
            ('action',       '60', 2023-06-12 17:32:31.985217, '0.25'),
            ('action',       '60', 2023-06-12 17:32:31.985217,  '0.5'),
            ('action',       '60', 2023-06-12 17:32:31.985217, '0.75'),
            ('action',       '60', 2023-06-12 17:32:31.985217,  '1.0'),
            ('action',       '70', 2023-06-12 17:32:32.005217,  '0.0'),
            ('action',       '70', 2023-06-12 17:32:32.005217, '0.25'),
            ('action',       '70', 2023-06-12 17:32:32.005217,  '0.5'),
            ('action',       '70', 2023-06-12 17:32:32.005217, '

tuple                                                                                           action   
measurement                                                                                         50   
timestep                                                                    2023-06-12 17:32:31.965217   
extra                                                                                              0.0   
vehicle driver        episodestart               timestamp                                               
MP73    wang-cheng    2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
        zheng-longfei 2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
VB7     wang-cheng    2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
                      2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
        zheng-longfei 2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   

tuple                                                                               
measurement                                                                         
timestep                                                                            
extra                                                                        0.25   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217  14.0  \
                                                 2023-06-12 19:32:31.865217  14.0   
                                                 2023-06-12 20:32:31.865217  14.0   
                                                 2023-06-12 21:32:31.865217  14.0   
                                                 2023-

## append new episode to the specific parquet file

In [232]:
dfs_episode4 = dfs_episode.copy()
dfs_episode4.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(8, 'd')], level='episodestart'
)
dfs_episode4.index = dfs_episode3.index.set_levels(
    [[trucks_by_name['VB4'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
dfs_episode4 = dfs_episode4.swaplevel(0, 1, axis=0).swaplevel(1, 2, axis=0)
dfs_episode4.columns

# dfs_episode5 = dfs_episode.copy()
# dfs_episode5.index = dfs_episode3.index.set_levels([episodestart-pd.Timedelta(9,'d')], level='episode')
df_episodes_read_multi_setindex.columns

MultiIndex([( 'state',    'brake', 2023-06-12 17:32:31.865217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.885217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.905217,   ''),
            ( 'state',    'brake', 2023-06-12 17:32:31.925217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.865217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.885217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.905217,   ''),
            ( 'state',   'thrust', 2023-06-12 17:32:31.925217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.865217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.885217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.905217,   ''),
            ( 'state', 'velocity', 2023-06-12 17:32:31.925217,   ''),
            ('reward',         '', 2023-06-12 17:32:36.865217,   ''),
            ('action',         50, 2023-06-12 17:32:31.965217,  0.0),
            ('action

MultiIndex([('action',       '50', 2023-06-12 17:32:31.965217,  '0.0'),
            ('action',       '50', 2023-06-12 17:32:31.965217, '0.25'),
            ('action',       '50', 2023-06-12 17:32:31.965217,  '0.5'),
            ('action',       '50', 2023-06-12 17:32:31.965217, '0.75'),
            ('action',       '50', 2023-06-12 17:32:31.965217,  '1.0'),
            ('action',       '60', 2023-06-12 17:32:31.985217,  '0.0'),
            ('action',       '60', 2023-06-12 17:32:31.985217, '0.25'),
            ('action',       '60', 2023-06-12 17:32:31.985217,  '0.5'),
            ('action',       '60', 2023-06-12 17:32:31.985217, '0.75'),
            ('action',       '60', 2023-06-12 17:32:31.985217,  '1.0'),
            ('action',       '70', 2023-06-12 17:32:32.005217,  '0.0'),
            ('action',       '70', 2023-06-12 17:32:32.005217, '0.25'),
            ('action',       '70', 2023-06-12 17:32:32.005217,  '0.5'),
            ('action',       '70', 2023-06-12 17:32:32.005217, '

In [223]:
dfs_episodes_new = pd.concat([df_episodes_read_multi_setindex, dfs_episode4], axis=0)
dfs_episodes_new
# dfs_episodes_new.columns
# dfs_episodes_new.index

tuple                                                                                           action   
measurement                                                                                         50   
timestep                                                                    2023-06-12 17:32:31.965217   
extra                                                                                              0.0   
vehicle driver        episodestart               timestamp                                               
MP73    wang-cheng    2023-06-07 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0  \
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
        zheng-longfei 2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
VB7     wang-cheng    2023-06-06 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
                      2023-06-12 17:32:31.865217 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
        zheng-longfei 2023-06-10 18:46:10.575552 2023-06-12 18:32:31.865217                       13.0   
                                                 2023-06-12 19:32:31.865217                       13.0   
                                                 2023-06-12 20:32:31.865217                       13.0   
                                                 2023-06-12 21:32:31.865217                       13.0   
                                                 2023-06-12 22:32:31.865217                       13.0   
VB4     zheng-longfei 2023-06-05 18:46:10.575552 2023-06-12 18:32:31.865217                        NaN   
                                                 2023-06-12 19:32:31.865217                        NaN   
                                                 2023-06-12 20:32:31.865217                        NaN   
                                                 2023-06-12 21:32:31.865217                        NaN   
                                                 2023-06-12 22:32:31.865217                        NaN   

tuple                                                                               
measurement                                                                         
timestep                                                                            
extra                             

#### save new dataframe back to parquet

In [184]:

os.getcwd()

'/home/n/devel/anting_remote/eos/data'

In [208]:
dfs_episodes_new_flat = dfs_episodes_new.copy()
dfs_episodes_new_flat = dfs_episodes_new_flat.reset_index(level=['vehicle', 'driver'])
dfs_episodes_new_flat.columns
dfs_episodes_new_flat.index

MultiIndex([('vehicle',         '',                        'NaT',     ''),
            ( 'driver',         '',                        'NaT',     ''),
            ( 'action',       '50', '2023-06-12 17:32:31.965217',  '0.0'),
            ( 'action',       '50', '2023-06-12 17:32:31.965217', '0.25'),
            ( 'action',       '50', '2023-06-12 17:32:31.965217',  '0.5'),
            ( 'action',       '50', '2023-06-12 17:32:31.965217', '0.75'),
            ( 'action',       '50', '2023-06-12 17:32:31.965217',  '1.0'),
            ( 'action',       '60', '2023-06-12 17:32:31.985217',  '0.0'),
            ( 'action',       '60', '2023-06-12 17:32:31.985217', '0.25'),
            ( 'action',       '60', '2023-06-12 17:32:31.985217',  '0.5'),
            ( 'action',       '60', '2023-06-12 17:32:31.985217', '0.75'),
            ( 'action',       '60', '2023-06-12 17:32:31.985217',  '1.0'),
            ( 'action',       '70', '2023-06-12 17:32:32.005217',  '0.0'),
            ( 'action',  

MultiIndex([('2023-06-07 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-07 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 21:32:31.865217'),
            ('2023-06-05 18:46:10.575552', '2023-06-12 22:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 18:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 19:32:31.865217'),
            ('2023-06-06 18:46:10.575552', '2023-06-12 20:32:31.865217'),
            ('2023-06-06 18:46:10.5755

In [216]:
# encoding index
# from datetime import datetime

flat_idx = dfs_episodes_new_flat.index.to_flat_index()
flat_idx = [
    f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[1].strftime("%Y-%m-%dT%H:%M:%S.%f")}' if x[1] != '' else f'{x[0].strftime("%Y-%m-%dT%H:%M:%S.%f")}'
    for x in flat_idx
    ]
# flat_idx

# encoding columns
flat_cols = dfs_episodes_new_flat.columns.to_flat_index()
# flat_cols
flat_cols = [
    f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_{x[3]}'
    if (x[1] != '' and x[3] not in ('', pd.NaT,str(pd.NA)))
    else f'{x[0]}_{x[1]}_{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[1] != '' and x[3] in ('',pd.NaT,str(pd.NA)))
    else f'{x[0]}__{x[2].strftime("%Y-%m-%dT%H:%M:%S.%f")}_'
    if (x[2] not in ('',pd.NaT,str(pd.NA)))
    else f'{x[0]}___'
    for x in flat_cols
]
flat_cols

['vehicle___',
 'driver___',
 'action_50_2023-06-12T17:32:31.965217_0.0',
 'action_50_2023-06-12T17:32:31.965217_0.25',
 'action_50_2023-06-12T17:32:31.965217_0.5',
 'action_50_2023-06-12T17:32:31.965217_0.75',
 'action_50_2023-06-12T17:32:31.965217_1.0',
 'action_60_2023-06-12T17:32:31.985217_0.0',
 'action_60_2023-06-12T17:32:31.985217_0.25',
 'action_60_2023-06-12T17:32:31.985217_0.5',
 'action_60_2023-06-12T17:32:31.985217_0.75',
 'action_60_2023-06-12T17:32:31.985217_1.0',
 'action_70_2023-06-12T17:32:32.005217_0.0',
 'action_70_2023-06-12T17:32:32.005217_0.25',
 'action_70_2023-06-12T17:32:32.005217_0.5',
 'action_70_2023-06-12T17:32:32.005217_0.75',
 'action_70_2023-06-12T17:32:32.005217_1.0',
 'nstate_brake_2023-06-12T17:32:36.865217_',
 'nstate_brake_2023-06-12T17:32:36.885217_',
 'nstate_brake_2023-06-12T17:32:36.905217_',
 'nstate_brake_2023-06-12T17:32:36.925217_',
 'nstate_thrust_2023-06-12T17:32:36.865217_',
 'nstate_thrust_2023-06-12T17:32:36.885217_',
 'nstate_thrust_20

In [219]:

dfs_episodes_new_flat.columns = flat_cols
dfs_episodes_new_flat.index = flat_idx
dfs_episodes_new_flat['vehicle___'] = dfs_episodes_new_flat['vehicle___'].astype("category")
dfs_episodes_new_flat['driver___'] = dfs_episodes_new_flat['driver___'].astype("category")

#dfs_episodes_new_flat.index.name = 'episodestart_timestamp'
dfs_episodes_new_flat.index
dfs_episodes_new_flat.dtypes
# dfs_episodes_new_flat.columns.name = 'episodestart_timestamp'
dfs_episodes_new_flat.columns
dfs_episodes_new_flat


Index(['2023-06-07T18:46:10.575552_2023-06-12T18:32:31.865217',
       '2023-06-07T18:46:10.575552_2023-06-12T19:32:31.865217',
       '2023-06-07T18:46:10.575552_2023-06-12T20:32:31.865217',
       '2023-06-07T18:46:10.575552_2023-06-12T21:32:31.865217',
       '2023-06-07T18:46:10.575552_2023-06-12T22:32:31.865217',
       '2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217',
       '2023-06-05T18:46:10.575552_2023-06-12T19:32:31.865217',
       '2023-06-05T18:46:10.575552_2023-06-12T20:32:31.865217',
       '2023-06-05T18:46:10.575552_2023-06-12T21:32:31.865217',
       '2023-06-05T18:46:10.575552_2023-06-12T22:32:31.865217',
       '2023-06-06T18:46:10.575552_2023-06-12T18:32:31.865217',
       '2023-06-06T18:46:10.575552_2023-06-12T19:32:31.865217',
       '2023-06-06T18:46:10.575552_2023-06-12T20:32:31.865217',
       '2023-06-06T18:46:10.575552_2023-06-12T21:32:31.865217',
       '2023-06-06T18:46:10.575552_2023-06-12T22:32:31.865217',
       '2023-06-12T17:32:31.865217_2023-

vehicle___                                     category
driver___                                      category
action_50_2023-06-12T17:32:31.965217_0.0        float64
action_50_2023-06-12T17:32:31.965217_0.25       float64
action_50_2023-06-12T17:32:31.965217_0.5        float64
action_50_2023-06-12T17:32:31.965217_0.75       float64
action_50_2023-06-12T17:32:31.965217_1.0        float64
action_60_2023-06-12T17:32:31.985217_0.0        float64
action_60_2023-06-12T17:32:31.985217_0.25       float64
action_60_2023-06-12T17:32:31.985217_0.5        float64
action_60_2023-06-12T17:32:31.985217_0.75       float64
action_60_2023-06-12T17:32:31.985217_1.0        float64
action_70_2023-06-12T17:32:32.005217_0.0        float64
action_70_2023-06-12T17:32:32.005217_0.25       float64
action_70_2023-06-12T17:32:32.005217_0.5        float64
action_70_2023-06-12T17:32:32.005217_0.75       float64
action_70_2023-06-12T17:32:32.005217_1.0        float64
nstate_brake_2023-06-12T17:32:36.865217_        

Index(['vehicle___', 'driver___', 'action_50_2023-06-12T17:32:31.965217_0.0',
       'action_50_2023-06-12T17:32:31.965217_0.25',
       'action_50_2023-06-12T17:32:31.965217_0.5',
       'action_50_2023-06-12T17:32:31.965217_0.75',
       'action_50_2023-06-12T17:32:31.965217_1.0',
       'action_60_2023-06-12T17:32:31.985217_0.0',
       'action_60_2023-06-12T17:32:31.985217_0.25',
       'action_60_2023-06-12T17:32:31.985217_0.5',
       'action_60_2023-06-12T17:32:31.985217_0.75',
       'action_60_2023-06-12T17:32:31.985217_1.0',
       'action_70_2023-06-12T17:32:32.005217_0.0',
       'action_70_2023-06-12T17:32:32.005217_0.25',
       'action_70_2023-06-12T17:32:32.005217_0.5',
       'action_70_2023-06-12T17:32:32.005217_0.75',
       'action_70_2023-06-12T17:32:32.005217_1.0',
       'nstate_brake_2023-06-12T17:32:36.865217_',
       'nstate_brake_2023-06-12T17:32:36.885217_',
       'nstate_brake_2023-06-12T17:32:36.905217_',
       'nstate_brake_2023-06-12T17:32:36.925217_'

,vehicle___,driver___,action_50_2023-06-12T17:32:31.965217_0.0,action_50_2023-06-12T17:32:31.965217_0.25,action_50_2023-06-12T17:32:31.965217_0.5,action_50_2023-06-12T17:32:31.965217_0.75,action_50_2023-06-12T17:32:31.965217_1.0,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,...,action_60_2023-06-12T17:32:31.985217_0.0,action_60_2023-06-12T17:32:31.985217_0.25,action_60_2023-06-12T17:32:31.985217_0.5,action_60_2023-06-12T17:32:31.985217_0.75,action_60_2023-06-12T17:32:31.985217_1.0,action_70_2023-06-12T17:32:32.005217_0.0,action_70_2023-06-12T17:32:32.005217_0.25,action_70_2023-06-12T17:32:32.005217_0.5,action_70_2023-06-12T17:32:32.005217_0.75,action_70_2023-06-12T17:32:32.005217_1.0
2023-06-07T18:46:10.575552_2023-06-12T18:32:31.865217,MP73,wang-cheng,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-07T18:46:10.575552_2023-06-12T19:32:31.865217,MP73,wang-cheng,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-07T18:46:10.575552_2023-06-12T20:32:31.865217,MP73,wang-cheng,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-07T18:46:10.575552_2023-06-12T21:32:31.865217,MP73,wang-cheng,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-07T18:46:10.575552_2023-06-12T22:32:31.865217,MP73,wang-cheng,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-05T18:46:10.575552_2023-06-12T18:32:31.865217,MP73,zheng-longfei,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-05T18:46:10.575552_2023-06-12T19:32:31.865217,MP73,zheng-longfei,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-05T18:46:10.575552_2023-06-12T20:32:31.865217,MP73,zheng-longfei,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-05T18:46:10.575552_2023-06-12T21:32:31.865217,MP73,zheng-longfei,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-05T18:46:10.575552_2023-06-12T22:32:31.865217,MP73,zheng-longfei,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
try:
    ddf_episodes_new = dd.from_pandas(dfs_episodes_new_flat, npartitions=1)
except Exception as e:
    print(e)

# dfs_episodes_new_flat


In [222]:
os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[6, 600, 5],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_episodes_new.to_parquet(
            'eos_dask_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle___', 'driver___'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)

'/home/n/devel/anting_remote/eos/data'

Duplicate column names found: ['vehicle___', 'driver___', 'action_50_2023-06-12T17:32:31.965217_0.0', 'action_50_2023-06-12T17:32:31.965217_0.25', 'action_50_2023-06-12T17:32:31.965217_0.5', 'action_50_2023-06-12T17:32:31.965217_0.75', 'action_50_2023-06-12T17:32:31.965217_1.0', 'action_60_2023-06-12T17:32:31.985217_0.0', 'action_60_2023-06-12T17:32:31.985217_0.25', 'action_60_2023-06-12T17:32:31.985217_0.5', 'action_60_2023-06-12T17:32:31.985217_0.75', 'action_60_2023-06-12T17:32:31.985217_1.0', 'action_70_2023-06-12T17:32:32.005217_0.0', 'action_70_2023-06-12T17:32:32.005217_0.25', 'action_70_2023-06-12T17:32:32.005217_0.5', 'action_70_2023-06-12T17:32:32.005217_0.75', 'action_70_2023-06-12T17:32:32.005217_1.0', 'nstate_brake_2023-06-12T17:32:36.865217_', 'nstate_brake_2023-06-12T17:32:36.885217_', 'nstate_brake_2023-06-12T17:32:36.905217_', 'nstate_brake_2023-06-12T17:32:36.925217_', 'nstate_thrust_2023-06-12T17:32:36.865217_', 'nstate_thrust_2023-06-12T17:32:36.885217_', 'nstate_th

In [ ]:
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('eos_episodes/_common_metadata')
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in df.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA


multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
multi_col

multi_idx = [tuple(idx.split('_')) for idx in df.index]
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
multi_idx
df.columns = multi_col
df.index = multi_idx

df = df.set_index(['vehicle', 'driver', df.index])
df
# len(multi_idx)

### add rows to a partition

In [ ]:

ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
ddf.index
ddf.npartitions
ddf.divisions
ddf.compute()

In [ ]:
dfs_episode5 = dfs_episode.copy()
dfs_episode5.index = dfs_episode4.index.set_levels(
    [episodestart - pd.Timedelta(9, 'd')], level='episodestart'
)
dfs_episode5.index = dfs_episode4.index.set_levels(
    [[trucks[1].vid], [drivers[0].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
# dfs_episode5 = dfs_episode5.swaplevel(0,1, axis=0).swaplevel(1,2,axis=0)
dfs_episode5 = dfs_episode5.reset_index(level=['vehicle', 'driver'])

dfs_episode5['vehicle'] = dfs_episode5['vehicle'].astype('category')
dfs_episode5['driver'] = dfs_episode5['driver'].astype('category')
dfs_episode5

#### decoding single dataframe of multiindex into dask single index dataframe

In [ ]:
df_episode_new = dfs_episode5.copy()
df_episode_new.columns = [
    f'{x[0]}_{x[1]}_{x[2]}'
    if (x[1] != '')
    else f'{x[0]}__{x[2]}'  # !!! dunder!!!
    if (x[2] != '')
    else f'{x[0]}'
    for x in dfs_episode5.columns.to_flat_index()
]
df_episode_new.index = [
    f'{x[0]}_{x[1]}' if x[1] != '' else f'x{0}'
    for x in dfs_episode5.index.to_flat_index()
]
# df_episode_new.dtypes
df_episode_new

In [ ]:

ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
df = ddf.compute()
df.dtypes

In [ ]:
df_list = []
for n in range(ddf.npartitions):
    df_list.append(ddf.partitions[n])
len(df_list)

In [ ]:
for i, ddf in enumerate(df_list):
    # if dfs_episode_new['vehicle']
    print(f"{i}")
    # df = ddf.compute()
    # ddf.columns
    # ddf.dtypes
    vehicle = ddf['vehicle'].compute()[0]
    driver = ddf['driver'].compute()[0]
    # df_episode_new = df_episode_new.append(ddf)
    if (
        df_episode_new['vehicle'][0] == vehicle
        and df_episode_new['driver'][0] == driver
    ):
        print(
            f'hit {i}{"st" if i==1 else "nd" if i==2 else "rd" if i==3 else "th"} partition'
        )
        df_list[i] = dd.concat([ddf, df_episode_new], axis=0)
        # append ddf to dfs_episode_new

    #     c_ep = df_episode_new['ep
    #     dfs_episode_new = dfs_episode_newappend(ddf)
    # print(f"splicing dfs_episode_new {c_ep} with


In [ ]:

ddf_all = dd.concat(df_list, axis=0, join='outer')
# ddf_all.compute()os.getcwd()
custom_metadata = {
    b'eos': b'{"length":[8, 800, 8],"timezeone":"sh","units":["kph","pct","pct"],"data_propensity":[50, 1, 4],"Dataset Description": "MP vehicle from TBox"}'
}

try:
    with ProgressBar():
        ddf_all.to_parquet(
            'eos_episodes',
            engine='pyarrow',
            compression='snappy',
            partition_on=['vehicle', 'driver'],
            write_metadata_file=True,
            custom_metadata=custom_metadata,
        )
except Exception as e:
    print(e)


In [ ]:
# ddf_all.compute()
ddf = dd.read_parquet(
    'eos_episodes',
    engine='pyarrow',
    compression='snappy',
    ignore_metadata_file=False,
    infer_divisions=True,
)
pq_meta = pq.read_metadata('ddf_episodes/_common_metadata')
# print(table_meta)
# print(table_meta.metadata[b'eos'])
custom_meta_info = pq_meta.metadata[b'eos']
custom_meta_info = json.loads(custom_meta_info)
# custom_meta_info

for key, val in custom_meta_info.items():
    print(f'{key}: {val}')

df = ddf.compute()

multi_tpl = [tuple(col.split('_')) for col in ddf.columns]
multi_col = pd.MultiIndex.from_tuples(multi_tpl)
i1 = multi_col.get_level_values(0)
i2 = multi_col.get_level_values(
    1
)  # must be null string instead of the default pd.NA or np.nan
i2 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else idx for idx in i2
]  # convert index of level 2 type to int and '' if NA
i3 = multi_col.get_level_values(
    2
)  # must be null string instead of the default pd.NA or np.nan
i3 = [
    '' if str(idx) in (str(pd.NA), 'nan', '') else int(idx) for idx in i3
]  # convert index of level 2 type to int and '' if NA

multi_col = pd.MultiIndex.from_arrays([i1, i2, i3])
multi_col.names = ['tuple', 'measurement', 'step']
df.columns = multi_col

multi_idx = [tuple(idx.split('_')) for idx in ddf.index]
df_episodes_read_multiidx = df_episodes_read.copy()
multi_idx = pd.MultiIndex.from_tuples(multi_idx)
multi_idx.names = ['episodestart', 'timestamp']
df.index = multi_idx
df = df.set_index(['vehicle', 'driver', df.index])
df
df.columns
df.index
# len(multi_idx)


In [ ]:
df = df.sort_index()
df